In [3]:
import pandas as pd
import numpy as np
from pycaret.regression import *
from scipy.stats import gaussian_kde

from utils import *
from descriptor import *
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor
from sklearn.metrics import r2_score

base_path = os.getcwd()
warnings.filterwarnings('ignore')
filename = "../data/features/soap_topology_total.csv"
data = pd.read_csv(filename,index_col=0)
data

,cif_file,topology,first_building_block,second_building_block_type,second_building_block,edge,f-chi-0-all,f-chi-1-all,f-chi-2-all,f-chi-3-all,f-Z-0-all,f-Z-1-all,f-Z-2-all,f-Z-3-all,f-I-0-all,f-I-1-all,f-I-2-all,f-I-3-all,f-T-0-all,f-T-1-all,f-T-2-all,f-T-3-all,f-S-0-all,f-S-1-all,f-S-2-all,f-S-3-all,mc-chi-0-all,mc-chi-1-all,mc-chi-2-all,mc-chi-3-all,mc-Z-0-all,mc-Z-1-all,mc-Z-2-all,mc-Z-3-all,mc-I-0-all,mc-I-1-all,mc-I-2-all,mc-I-3-all,mc-T-0-all,mc-T-1-all,mc-T-2-all,mc-T-3-all,mc-S-0-all,mc-S-1-all,mc-S-2-all,mc-S-3-all,D_mc-chi-0-all,D_mc-chi-1-all,D_mc-chi-2-all,D_mc-chi-3-all,D_mc-Z-0-all,D_mc-Z-1-all,D_mc-Z-2-all,D_mc-Z-3-all,D_mc-I-0-all,D_mc-I-1-all,D_mc-I-2-all,D_mc-I-3-all,D_mc-T-0-all,D_mc-T-1-all,D_mc-T-2-all,D_mc-T-3-all,D_mc-S-0-all,D_mc-S-1-all,D_mc-S-2-all,D_mc-S-3-all,f-lig-chi-0,f-lig-chi-1,f-lig-chi-2,f-lig-chi-3,f-lig-Z-0,f-lig-Z-1,f-lig-Z-2,f-lig-Z-3,f-lig-I-0,f-lig-I-1,f-lig-I-2,f-lig-I-3,f-lig-T-0,f-lig-T-1,f-lig-T-2,f-lig-T-3,f-lig-S-0,f-lig-S-1,f-lig-S-2,f-lig-S-3,lc-chi-0-all,lc-chi-1-all,lc-chi-2-all,lc-chi-3-all,lc-Z-0-all,lc-Z-1-all,lc-Z-2-all,lc-Z-3-all,lc-I-0-all,lc-I-1-all,lc-I-2-all,lc-I-3-all,lc-T-0-all,lc-T-1-all,lc-T-2-all,lc-T-3-all,lc-S-0-all,lc-S-1-all,lc-S-2-all,lc-S-3-all,lc-alpha-0-all,lc-alpha-1-all,lc-alpha-2-all,lc-alpha-3-all,D_lc-chi-0-all,D_lc-chi-1-all,D_lc-chi-2-all,D_lc-chi-3-all,D_lc-Z-0-all,D_lc-Z-1-all,D_lc-Z-2-all,D_lc-Z-3-all,D_lc-I-0-all,D_lc-I-1-all,D_lc-I-2-all,D_lc-I-3-all,D_lc-T-0-all,D_lc-T-1-all,D_lc-T-2-all,D_lc-T-3-all,D_lc-S-0-all,D_lc-S-1-all,D_lc-S-2-all,D_lc-S-3-all,D_lc-alpha-0-all,D_lc-alpha-1-all,D_lc-alpha-2-all,D_lc-alpha-3-all,func-chi-0-all,func-chi-1-all,func-chi-2-all,func-chi-3-all,func-Z-0-all,func-Z-1-all,func-Z-2-all,func-Z-3-all,func-I-0-all,func-I-1-all,func-I-2-all,func-I-3-all,func-T-0-all,func-T-1-all,func-T-2-all,func-T-3-all,func-S-0-all,func-S-1-all,func-S-2-all,func-S-3-all,func-alpha-0-all,func-alpha-1-all,func-alpha-2-all,func-alpha-3-all,D_func-chi-0-all,D_func-chi-1-all,D_func-chi-2-all,D_func-chi-3-all,D_func-Z-0-all,D_func-Z-1-all,D_func-Z-2-all,D_func-Z-3-all,D_func-I-0-all,D_func-I-1-all,D_func-I-2-all,D_func-I-3-all,D_func-T-0-all,D_func-T-1-all,D_func-T-2-all,D_func-T-3-all,D_func-S-0-all,D_func-S-1-all,D_func-S-2-all,D_func-S-3-all,D_func-alpha-0-all,D_func-alpha-1-all,D_func-alpha-2-all,D_func-alpha-3-all,Di,Df,Dif,rho,VSA,GSA,VPOV,GPOV,POAV_vol_frac,PONAV_vol_frac,GPOAV,GPONAV,POAV,PONAV,KVRH,GVRH,soap0,soap1,soap2,soap3,soap4,soap5,soap6,soap7,soap8,soap9,soap10,soap11,soap12,soap13,soap14,soap15,soap16,soap17,soap18,soap19,soap20,soap21,soap22,soap23,soap24,soap25,soap26,soap27,soap28,soap29,soap30,soap31,soap32,soap33,soap34,soap35,soap36,soap37,soap38,soap39,soap40,soap41,soap42,soap43,soap44,soap45,soap46,soap47,soap48,soap49,soap50,soap51,...,soap7310,soap7311,soap7312,soap7313,soap7314,soap7315,soap7316,soap7317,soap7318,soap7319,soap7320,soap7321,soap7322,soap7323,soap7324,soap7325,soap7326,soap7327,soap7328,soap7329,soap7330,soap7331,soap7332,soap7333,soap7334,soap7335,soap7336,soap7337,soap7338,soap7339,soap7340,soap7341,soap7342,soap7343,soap7344,soap7345,soap7346,soap7347,soap7348,soap7349,soap7350,soap7351,soap7352,soap7353,soap7354,soap7355,soap7356,soap7357,soap7358,soap7359,soap7360,soap7361,soap7362,soap7363,soap7364,soap7365,soap7366,soap7367,soap7368,soap7369,soap7370,soap7371,soap7372,soap7373,soap7374,soap7375,soap7376,soap7377,soap7378,soap7379,soap7380,soap7381,soap7382,soap7383,soap7384,soap7385,soap7386,soap7387,soap7388,soap7389,soap7390,soap7391,soap7392,soap7393,soap7394,soap7395,soap7396,soap7397,soap7398,soap7399,soap7400,soap7401,soap7402,soap7403,soap7404,soap7405,soap7406,soap7407,soap7408,soap7409,soap7410,soap7411,soap7412,soap7413,soap7414,soap7415,soap7416,soap7417,soap7418,soap7419,soap7420,soap7421,soap7422,soap7423,soap7424,soap7425,soap7426,soap7427,soap7428,soap7429,soap7430,soap7431,soap7432,soap7433,soap7434,soap7435,soap7436,soap7437,soap7438,soap7439,soap7440,soap7441,soap7442,

In [4]:
soap_columns = list(data.columns)[198:]

columns = geometric + RAC + soap_columns + property_
new_columns = geometric + RAC + soap_columns
descriptors = data[new_columns]

drop_col = remove_constant_value_features(descriptors)
new_df_columns = [e for e in descriptors.columns if e not in drop_col]

property_ = ["KVRH"]

X = data[new_df_columns]
y = data[property_]

In [5]:
train_size = 1000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)



Learning rate set to 0.040943
0:	learn: 5.2603605	total: 146ms	remaining: 2m 25s
1:	learn: 5.1744257	total: 205ms	remaining: 1m 42s
2:	learn: 5.1094457	total: 264ms	remaining: 1m 27s
3:	learn: 5.0228660	total: 323ms	remaining: 1m 20s
4:	learn: 4.9410735	total: 382ms	remaining: 1m 15s
5:	learn: 4.8559687	total: 441ms	remaining: 1m 13s
6:	learn: 4.7733399	total: 499ms	remaining: 1m 10s
7:	learn: 4.6988588	total: 557ms	remaining: 1m 9s
8:	learn: 4.6290239	total: 616ms	remaining: 1m 7s
9:	learn: 4.5560197	total: 676ms	remaining: 1m 6s
10:	learn: 4.4880927	total: 735ms	remaining: 1m 6s
11:	learn: 4.4137695	total: 794ms	remaining: 1m 5s
12:	learn: 4.3575463	total: 852ms	remaining: 1m 4s
13:	learn: 4.3021472	total: 911ms	remaining: 1m 4s
14:	learn: 4.2498736	total: 971ms	remaining: 1m 3s
15:	learn: 4.1917564	total: 1.03s	remaining: 1m 3s
16:	learn: 4.1399783	total: 1.09s	remaining: 1m 2s
17:	learn: 4.0844140	total: 1.15s	remaining: 1m 2s
18:	learn: 4.0243529	total: 1.21s	remaining: 1m 2s
19:	

161:	learn: 1.8635694	total: 9.62s	remaining: 49.8s
162:	learn: 1.8607782	total: 9.68s	remaining: 49.7s
163:	learn: 1.8579336	total: 9.73s	remaining: 49.6s
164:	learn: 1.8515362	total: 9.79s	remaining: 49.6s
165:	learn: 1.8491913	total: 9.85s	remaining: 49.5s
166:	learn: 1.8420018	total: 9.91s	remaining: 49.4s
167:	learn: 1.8393572	total: 9.97s	remaining: 49.4s
168:	learn: 1.8342186	total: 10s	remaining: 49.3s
169:	learn: 1.8284048	total: 10.1s	remaining: 49.2s
170:	learn: 1.8229417	total: 10.1s	remaining: 49.2s
171:	learn: 1.8211736	total: 10.2s	remaining: 49.1s
172:	learn: 1.8164761	total: 10.3s	remaining: 49.1s
173:	learn: 1.8111206	total: 10.3s	remaining: 49s
174:	learn: 1.8080263	total: 10.4s	remaining: 48.9s
175:	learn: 1.8026992	total: 10.4s	remaining: 48.9s
176:	learn: 1.8002181	total: 10.5s	remaining: 48.8s
177:	learn: 1.7943108	total: 10.6s	remaining: 48.7s
178:	learn: 1.7860289	total: 10.6s	remaining: 48.7s
179:	learn: 1.7819445	total: 10.7s	remaining: 48.6s
180:	learn: 1.77

321:	learn: 1.1551306	total: 19s	remaining: 40s
322:	learn: 1.1519460	total: 19.1s	remaining: 40s
323:	learn: 1.1476485	total: 19.1s	remaining: 39.9s
324:	learn: 1.1438780	total: 19.2s	remaining: 39.8s
325:	learn: 1.1397913	total: 19.2s	remaining: 39.8s
326:	learn: 1.1372560	total: 19.3s	remaining: 39.7s
327:	learn: 1.1341605	total: 19.4s	remaining: 39.7s
328:	learn: 1.1331853	total: 19.4s	remaining: 39.6s
329:	learn: 1.1321255	total: 19.5s	remaining: 39.5s
330:	learn: 1.1313059	total: 19.5s	remaining: 39.5s
331:	learn: 1.1281404	total: 19.6s	remaining: 39.4s
332:	learn: 1.1219270	total: 19.6s	remaining: 39.4s
333:	learn: 1.1191125	total: 19.7s	remaining: 39.3s
334:	learn: 1.1159139	total: 19.8s	remaining: 39.2s
335:	learn: 1.1137198	total: 19.8s	remaining: 39.2s
336:	learn: 1.1096114	total: 19.9s	remaining: 39.1s
337:	learn: 1.1051735	total: 19.9s	remaining: 39.1s
338:	learn: 1.1014785	total: 20s	remaining: 39s
339:	learn: 1.1002308	total: 20.1s	remaining: 38.9s
340:	learn: 1.0969809	

481:	learn: 0.7728306	total: 28.4s	remaining: 30.5s
482:	learn: 0.7697991	total: 28.4s	remaining: 30.4s
483:	learn: 0.7678009	total: 28.5s	remaining: 30.4s
484:	learn: 0.7665065	total: 28.6s	remaining: 30.3s
485:	learn: 0.7653293	total: 28.6s	remaining: 30.3s
486:	learn: 0.7631007	total: 28.7s	remaining: 30.2s
487:	learn: 0.7623174	total: 28.7s	remaining: 30.1s
488:	learn: 0.7599689	total: 28.8s	remaining: 30.1s
489:	learn: 0.7577292	total: 28.8s	remaining: 30s
490:	learn: 0.7571491	total: 28.9s	remaining: 30s
491:	learn: 0.7551911	total: 29s	remaining: 29.9s
492:	learn: 0.7510542	total: 29s	remaining: 29.8s
493:	learn: 0.7490916	total: 29.1s	remaining: 29.8s
494:	learn: 0.7478724	total: 29.1s	remaining: 29.7s
495:	learn: 0.7461372	total: 29.2s	remaining: 29.7s
496:	learn: 0.7457344	total: 29.3s	remaining: 29.6s
497:	learn: 0.7451847	total: 29.3s	remaining: 29.5s
498:	learn: 0.7444487	total: 29.4s	remaining: 29.5s
499:	learn: 0.7426551	total: 29.4s	remaining: 29.4s
500:	learn: 0.742261

641:	learn: 0.5539347	total: 37.8s	remaining: 21.1s
642:	learn: 0.5536598	total: 37.8s	remaining: 21s
643:	learn: 0.5530299	total: 37.9s	remaining: 20.9s
644:	learn: 0.5516580	total: 37.9s	remaining: 20.9s
645:	learn: 0.5499737	total: 38s	remaining: 20.8s
646:	learn: 0.5488814	total: 38.1s	remaining: 20.8s
647:	learn: 0.5486090	total: 38.1s	remaining: 20.7s
648:	learn: 0.5467277	total: 38.2s	remaining: 20.6s
649:	learn: 0.5454463	total: 38.2s	remaining: 20.6s
650:	learn: 0.5435288	total: 38.3s	remaining: 20.5s
651:	learn: 0.5422204	total: 38.3s	remaining: 20.5s
652:	learn: 0.5415173	total: 38.4s	remaining: 20.4s
653:	learn: 0.5400115	total: 38.5s	remaining: 20.4s
654:	learn: 0.5383314	total: 38.5s	remaining: 20.3s
655:	learn: 0.5360790	total: 38.6s	remaining: 20.2s
656:	learn: 0.5347339	total: 38.6s	remaining: 20.2s
657:	learn: 0.5337588	total: 38.7s	remaining: 20.1s
658:	learn: 0.5333078	total: 38.8s	remaining: 20.1s
659:	learn: 0.5309402	total: 38.8s	remaining: 20s
660:	learn: 0.5298

...

327:	learn: 1.2836485	total: 19.4s	remaining: 39.7s
328:	learn: 1.2801027	total: 19.4s	remaining: 39.6s
329:	learn: 1.2729581	total: 19.5s	remaining: 39.6s
330:	learn: 1.2666644	total: 19.5s	remaining: 39.5s
331:	learn: 1.2628918	total: 19.6s	remaining: 39.4s
332:	learn: 1.2581075	total: 19.7s	remaining: 39.4s
333:	learn: 1.2545742	total: 19.7s	remaining: 39.3s
334:	learn: 1.2475144	total: 19.8s	remaining: 39.3s
335:	learn: 1.2442325	total: 19.8s	remaining: 39.2s
336:	learn: 1.2433512	total: 19.9s	remaining: 39.1s
337:	learn: 1.2361548	total: 20s	remaining: 39.1s
338:	learn: 1.2331223	total: 20s	remaining: 39s
339:	learn: 1.2303736	total: 20.1s	remaining: 39s
340:	learn: 1.2216879	total: 20.1s	remaining: 38.9s
341:	learn: 1.2180436	total: 20.2s	remaining: 38.8s
342:	learn: 1.2125131	total: 20.3s	remaining: 38.8s
343:	learn: 1.2094463	total: 20.3s	remaining: 38.7s
344:	learn: 1.2078482	total: 20.4s	remaining: 38.7s
345:	learn: 1.2017823	total: 20.4s	remaining: 38.6s
346:	learn: 1.198286

487:	learn: 0.8295797	total: 28.8s	remaining: 30.2s
488:	learn: 0.8274722	total: 28.9s	remaining: 30.2s
489:	learn: 0.8246120	total: 28.9s	remaining: 30.1s
490:	learn: 0.8234464	total: 29s	remaining: 30s
491:	learn: 0.8224420	total: 29s	remaining: 30s
492:	learn: 0.8188798	total: 29.1s	remaining: 29.9s
493:	learn: 0.8161749	total: 29.2s	remaining: 29.9s
494:	learn: 0.8132212	total: 29.2s	remaining: 29.8s
495:	learn: 0.8104988	total: 29.3s	remaining: 29.8s
496:	learn: 0.8096928	total: 29.3s	remaining: 29.7s
497:	learn: 0.8070377	total: 29.4s	remaining: 29.6s
498:	learn: 0.8058033	total: 29.5s	remaining: 29.6s
499:	learn: 0.8053256	total: 29.5s	remaining: 29.5s
500:	learn: 0.8042607	total: 29.6s	remaining: 29.5s
501:	learn: 0.8032111	total: 29.6s	remaining: 29.4s
502:	learn: 0.7996764	total: 29.7s	remaining: 29.3s
503:	learn: 0.7980616	total: 29.8s	remaining: 29.3s
504:	learn: 0.7960346	total: 29.8s	remaining: 29.2s
505:	learn: 0.7943129	total: 29.9s	remaining: 29.2s
506:	learn: 0.791156

647:	learn: 0.5735915	total: 38.3s	remaining: 20.8s
648:	learn: 0.5725243	total: 38.3s	remaining: 20.7s
649:	learn: 0.5697579	total: 38.4s	remaining: 20.7s
650:	learn: 0.5680239	total: 38.4s	remaining: 20.6s
651:	learn: 0.5674752	total: 38.5s	remaining: 20.6s
652:	learn: 0.5670604	total: 38.6s	remaining: 20.5s
653:	learn: 0.5663408	total: 38.6s	remaining: 20.4s
654:	learn: 0.5647316	total: 38.7s	remaining: 20.4s
655:	learn: 0.5641056	total: 38.7s	remaining: 20.3s
656:	learn: 0.5625256	total: 38.8s	remaining: 20.3s
657:	learn: 0.5613020	total: 38.9s	remaining: 20.2s
658:	learn: 0.5593742	total: 38.9s	remaining: 20.1s
659:	learn: 0.5584087	total: 39s	remaining: 20.1s
660:	learn: 0.5564388	total: 39s	remaining: 20s
661:	learn: 0.5557408	total: 39.1s	remaining: 20s
662:	learn: 0.5540574	total: 39.2s	remaining: 19.9s
663:	learn: 0.5534652	total: 39.2s	remaining: 19.8s
664:	learn: 0.5519615	total: 39.3s	remaining: 19.8s
665:	learn: 0.5503154	total: 39.3s	remaining: 19.7s
666:	learn: 0.548898

807:	learn: 0.4035684	total: 47.7s	remaining: 11.3s
808:	learn: 0.4025548	total: 47.8s	remaining: 11.3s
809:	learn: 0.4015321	total: 47.8s	remaining: 11.2s
810:	learn: 0.4002741	total: 47.9s	remaining: 11.2s
811:	learn: 0.3987971	total: 48s	remaining: 11.1s
812:	learn: 0.3986119	total: 48s	remaining: 11s
813:	learn: 0.3974799	total: 48.1s	remaining: 11s
814:	learn: 0.3961752	total: 48.1s	remaining: 10.9s
815:	learn: 0.3947835	total: 48.2s	remaining: 10.9s
816:	learn: 0.3941629	total: 48.3s	remaining: 10.8s
817:	learn: 0.3933961	total: 48.3s	remaining: 10.8s
818:	learn: 0.3932320	total: 48.4s	remaining: 10.7s
819:	learn: 0.3922784	total: 48.4s	remaining: 10.6s
820:	learn: 0.3906056	total: 48.5s	remaining: 10.6s
821:	learn: 0.3903159	total: 48.6s	remaining: 10.5s
822:	learn: 0.3901636	total: 48.6s	remaining: 10.5s
823:	learn: 0.3892114	total: 48.7s	remaining: 10.4s
824:	learn: 0.3888335	total: 48.7s	remaining: 10.3s
825:	learn: 0.3876752	total: 48.8s	remaining: 10.3s
826:	learn: 0.387498

967:	learn: 0.2890070	total: 57.2s	remaining: 1.89s
968:	learn: 0.2888931	total: 57.2s	remaining: 1.83s
969:	learn: 0.2885244	total: 57.3s	remaining: 1.77s
970:	learn: 0.2881554	total: 57.4s	remaining: 1.71s
971:	learn: 0.2871120	total: 57.4s	remaining: 1.65s
972:	learn: 0.2869163	total: 57.5s	remaining: 1.59s
973:	learn: 0.2866486	total: 57.5s	remaining: 1.53s
974:	learn: 0.2860048	total: 57.6s	remaining: 1.48s
975:	learn: 0.2852722	total: 57.7s	remaining: 1.42s
976:	learn: 0.2846603	total: 57.7s	remaining: 1.36s
977:	learn: 0.2835991	total: 57.8s	remaining: 1.3s
978:	learn: 0.2830548	total: 57.8s	remaining: 1.24s
979:	learn: 0.2822490	total: 57.9s	remaining: 1.18s
980:	learn: 0.2816070	total: 58s	remaining: 1.12s
981:	learn: 0.2808551	total: 58s	remaining: 1.06s
982:	learn: 0.2797907	total: 58.1s	remaining: 1s
983:	learn: 0.2790642	total: 58.1s	remaining: 945ms
984:	learn: 0.2778312	total: 58.2s	remaining: 886ms
985:	learn: 0.2766784	total: 58.3s	remaining: 827ms
986:	learn: 0.276429

In [6]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

1.5820995636327158
1.5874231238440124
1.559531146673001
1.622395409799905
1.5779064763821087
1.5632416614645521
0.6599677965105077
0.6369685212994579
0.6565010130210469
0.6457057745589476
0.6761521304244034
0.684511543248683


In [8]:
train_size = 2000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)


Learning rate set to 0.045682
0:	learn: 4.9050640	total: 107ms	remaining: 1m 46s
1:	learn: 4.8050335	total: 187ms	remaining: 1m 33s
2:	learn: 4.7135003	total: 267ms	remaining: 1m 28s
3:	learn: 4.6348390	total: 346ms	remaining: 1m 26s
4:	learn: 4.5578824	total: 425ms	remaining: 1m 24s
5:	learn: 4.4741953	total: 504ms	remaining: 1m 23s
6:	learn: 4.3982166	total: 584ms	remaining: 1m 22s
7:	learn: 4.3329542	total: 663ms	remaining: 1m 22s
8:	learn: 4.2682452	total: 742ms	remaining: 1m 21s
9:	learn: 4.2016856	total: 822ms	remaining: 1m 21s
10:	learn: 4.1232952	total: 902ms	remaining: 1m 21s
11:	learn: 4.0601550	total: 982ms	remaining: 1m 20s
12:	learn: 4.0000684	total: 1.06s	remaining: 1m 20s
13:	learn: 3.9444365	total: 1.14s	remaining: 1m 20s
14:	learn: 3.9005996	total: 1.22s	remaining: 1m 20s
15:	learn: 3.8400618	total: 1.3s	remaining: 1m 19s
16:	learn: 3.7857814	total: 1.38s	remaining: 1m 19s
17:	learn: 3.7379470	total: 1.46s	remaining: 1m 19s
18:	learn: 3.6931783	total: 1.54s	remaining: 

158:	learn: 2.0909158	total: 12.6s	remaining: 1m 6s
159:	learn: 2.0849194	total: 12.7s	remaining: 1m 6s
160:	learn: 2.0759724	total: 12.8s	remaining: 1m 6s
161:	learn: 2.0681886	total: 12.9s	remaining: 1m 6s
162:	learn: 2.0631454	total: 12.9s	remaining: 1m 6s
163:	learn: 2.0553908	total: 13s	remaining: 1m 6s
164:	learn: 2.0488870	total: 13.1s	remaining: 1m 6s
165:	learn: 2.0416889	total: 13.2s	remaining: 1m 6s
166:	learn: 2.0385386	total: 13.3s	remaining: 1m 6s
167:	learn: 2.0293846	total: 13.3s	remaining: 1m 6s
168:	learn: 2.0220575	total: 13.4s	remaining: 1m 5s
169:	learn: 2.0155364	total: 13.5s	remaining: 1m 5s
170:	learn: 2.0133464	total: 13.6s	remaining: 1m 5s
171:	learn: 2.0080286	total: 13.7s	remaining: 1m 5s
172:	learn: 2.0026127	total: 13.7s	remaining: 1m 5s
173:	learn: 1.9937299	total: 13.8s	remaining: 1m 5s
174:	learn: 1.9904293	total: 13.9s	remaining: 1m 5s
175:	learn: 1.9855245	total: 14s	remaining: 1m 5s
176:	learn: 1.9832239	total: 14.1s	remaining: 1m 5s
177:	learn: 1.98

320:	learn: 1.3931534	total: 25.5s	remaining: 53.9s
321:	learn: 1.3910101	total: 25.5s	remaining: 53.8s
322:	learn: 1.3869633	total: 25.6s	remaining: 53.7s
323:	learn: 1.3814165	total: 25.7s	remaining: 53.6s
324:	learn: 1.3788043	total: 25.8s	remaining: 53.5s
325:	learn: 1.3754900	total: 25.9s	remaining: 53.5s
326:	learn: 1.3710190	total: 25.9s	remaining: 53.4s
327:	learn: 1.3677330	total: 26s	remaining: 53.3s
328:	learn: 1.3659250	total: 26.1s	remaining: 53.2s
329:	learn: 1.3627909	total: 26.2s	remaining: 53.1s
330:	learn: 1.3602007	total: 26.3s	remaining: 53.1s
331:	learn: 1.3569790	total: 26.3s	remaining: 53s
332:	learn: 1.3557166	total: 26.4s	remaining: 52.9s
333:	learn: 1.3540259	total: 26.5s	remaining: 52.8s
334:	learn: 1.3528256	total: 26.6s	remaining: 52.7s
335:	learn: 1.3505348	total: 26.6s	remaining: 52.7s
336:	learn: 1.3456689	total: 26.7s	remaining: 52.6s
337:	learn: 1.3446612	total: 26.8s	remaining: 52.5s
338:	learn: 1.3398619	total: 26.9s	remaining: 52.4s
339:	learn: 1.33

479:	learn: 1.0236949	total: 38.1s	remaining: 41.3s
480:	learn: 1.0211840	total: 38.2s	remaining: 41.2s
481:	learn: 1.0180726	total: 38.2s	remaining: 41.1s
482:	learn: 1.0174320	total: 38.3s	remaining: 41s
483:	learn: 1.0156764	total: 38.4s	remaining: 40.9s
484:	learn: 1.0143872	total: 38.5s	remaining: 40.9s
485:	learn: 1.0127642	total: 38.6s	remaining: 40.8s
486:	learn: 1.0116753	total: 38.6s	remaining: 40.7s
487:	learn: 1.0098186	total: 38.7s	remaining: 40.6s
488:	learn: 1.0076908	total: 38.8s	remaining: 40.5s
489:	learn: 1.0057553	total: 38.9s	remaining: 40.5s
490:	learn: 1.0042543	total: 38.9s	remaining: 40.4s
491:	learn: 1.0012661	total: 39s	remaining: 40.3s
492:	learn: 0.9979325	total: 39.1s	remaining: 40.2s
493:	learn: 0.9970431	total: 39.2s	remaining: 40.1s
494:	learn: 0.9964201	total: 39.3s	remaining: 40.1s
495:	learn: 0.9955395	total: 39.3s	remaining: 40s
496:	learn: 0.9948578	total: 39.4s	remaining: 39.9s
497:	learn: 0.9929535	total: 39.5s	remaining: 39.8s
498:	learn: 0.9912

638:	learn: 0.7988515	total: 50.7s	remaining: 28.6s
639:	learn: 0.7983403	total: 50.7s	remaining: 28.5s
640:	learn: 0.7964642	total: 50.8s	remaining: 28.5s
641:	learn: 0.7962211	total: 50.9s	remaining: 28.4s
642:	learn: 0.7949609	total: 51s	remaining: 28.3s
643:	learn: 0.7931670	total: 51.1s	remaining: 28.2s
644:	learn: 0.7926162	total: 51.1s	remaining: 28.1s
645:	learn: 0.7921985	total: 51.2s	remaining: 28.1s
646:	learn: 0.7903112	total: 51.3s	remaining: 28s
647:	learn: 0.7900819	total: 51.4s	remaining: 27.9s
648:	learn: 0.7890739	total: 51.5s	remaining: 27.8s
649:	learn: 0.7872195	total: 51.5s	remaining: 27.8s
650:	learn: 0.7849801	total: 51.6s	remaining: 27.7s
651:	learn: 0.7845906	total: 51.7s	remaining: 27.6s
652:	learn: 0.7843797	total: 51.8s	remaining: 27.5s
653:	learn: 0.7823976	total: 51.9s	remaining: 27.4s
654:	learn: 0.7816246	total: 51.9s	remaining: 27.4s
655:	learn: 0.7813970	total: 52s	remaining: 27.3s
656:	learn: 0.7798132	total: 52.1s	remaining: 27.2s
657:	learn: 0.7782

...

305:	learn: 1.4723717	total: 24.3s	remaining: 55.1s
306:	learn: 1.4674027	total: 24.4s	remaining: 55s
307:	learn: 1.4666416	total: 24.4s	remaining: 54.9s
308:	learn: 1.4635921	total: 24.5s	remaining: 54.8s
309:	learn: 1.4597208	total: 24.6s	remaining: 54.8s
310:	learn: 1.4564140	total: 24.7s	remaining: 54.7s
311:	learn: 1.4556986	total: 24.8s	remaining: 54.6s
312:	learn: 1.4511154	total: 24.8s	remaining: 54.5s
313:	learn: 1.4504233	total: 24.9s	remaining: 54.4s
314:	learn: 1.4460179	total: 25s	remaining: 54.4s
315:	learn: 1.4426803	total: 25.1s	remaining: 54.3s
316:	learn: 1.4420158	total: 25.2s	remaining: 54.2s
317:	learn: 1.4380714	total: 25.2s	remaining: 54.1s
318:	learn: 1.4348273	total: 25.3s	remaining: 54s
319:	learn: 1.4341730	total: 25.4s	remaining: 54s
320:	learn: 1.4312770	total: 25.5s	remaining: 53.9s
321:	learn: 1.4282892	total: 25.6s	remaining: 53.8s
322:	learn: 1.4265855	total: 25.6s	remaining: 53.7s
323:	learn: 1.4216341	total: 25.7s	remaining: 53.6s
324:	learn: 1.416916

464:	learn: 1.1111337	total: 36.9s	remaining: 42.4s
465:	learn: 1.1083329	total: 37s	remaining: 42.4s
466:	learn: 1.1061811	total: 37s	remaining: 42.3s
467:	learn: 1.1057270	total: 37.1s	remaining: 42.2s
468:	learn: 1.1030223	total: 37.2s	remaining: 42.1s
469:	learn: 1.1011094	total: 37.3s	remaining: 42s
470:	learn: 1.0992204	total: 37.4s	remaining: 42s
471:	learn: 1.0982276	total: 37.4s	remaining: 41.9s
472:	learn: 1.0961948	total: 37.5s	remaining: 41.8s
473:	learn: 1.0955862	total: 37.6s	remaining: 41.7s
474:	learn: 1.0933831	total: 37.7s	remaining: 41.6s
475:	learn: 1.0926591	total: 37.7s	remaining: 41.6s
476:	learn: 1.0914197	total: 37.8s	remaining: 41.5s
477:	learn: 1.0890154	total: 37.9s	remaining: 41.4s
478:	learn: 1.0875262	total: 38s	remaining: 41.3s
479:	learn: 1.0857358	total: 38.1s	remaining: 41.2s
480:	learn: 1.0852788	total: 38.1s	remaining: 41.2s
481:	learn: 1.0833689	total: 38.2s	remaining: 41.1s
482:	learn: 1.0802268	total: 38.3s	remaining: 41s
483:	learn: 1.0792583	to

623:	learn: 0.8562249	total: 49.5s	remaining: 29.8s
624:	learn: 0.8551792	total: 49.6s	remaining: 29.7s
625:	learn: 0.8537943	total: 49.6s	remaining: 29.7s
626:	learn: 0.8526925	total: 49.7s	remaining: 29.6s
627:	learn: 0.8518994	total: 49.8s	remaining: 29.5s
628:	learn: 0.8503872	total: 49.9s	remaining: 29.4s
629:	learn: 0.8495971	total: 49.9s	remaining: 29.3s
630:	learn: 0.8493559	total: 50s	remaining: 29.3s
631:	learn: 0.8484101	total: 50.1s	remaining: 29.2s
632:	learn: 0.8480537	total: 50.2s	remaining: 29.1s
633:	learn: 0.8463751	total: 50.3s	remaining: 29s
634:	learn: 0.8453271	total: 50.3s	remaining: 28.9s
635:	learn: 0.8439341	total: 50.4s	remaining: 28.9s
636:	learn: 0.8426838	total: 50.5s	remaining: 28.8s
637:	learn: 0.8406082	total: 50.6s	remaining: 28.7s
638:	learn: 0.8388466	total: 50.7s	remaining: 28.6s
639:	learn: 0.8386150	total: 50.7s	remaining: 28.5s
640:	learn: 0.8378800	total: 50.8s	remaining: 28.5s
641:	learn: 0.8367720	total: 50.9s	remaining: 28.4s
642:	learn: 0.83

785:	learn: 0.6793934	total: 1m 2s	remaining: 17s
786:	learn: 0.6789843	total: 1m 2s	remaining: 16.9s
787:	learn: 0.6780143	total: 1m 2s	remaining: 16.8s
788:	learn: 0.6778481	total: 1m 2s	remaining: 16.7s
789:	learn: 0.6775346	total: 1m 2s	remaining: 16.6s
790:	learn: 0.6764083	total: 1m 2s	remaining: 16.6s
791:	learn: 0.6749092	total: 1m 2s	remaining: 16.5s
792:	learn: 0.6744024	total: 1m 2s	remaining: 16.4s
793:	learn: 0.6734356	total: 1m 2s	remaining: 16.3s
794:	learn: 0.6719723	total: 1m 3s	remaining: 16.3s
795:	learn: 0.6710872	total: 1m 3s	remaining: 16.2s
796:	learn: 0.6706116	total: 1m 3s	remaining: 16.1s
797:	learn: 0.6694408	total: 1m 3s	remaining: 16s
798:	learn: 0.6687074	total: 1m 3s	remaining: 15.9s
799:	learn: 0.6675820	total: 1m 3s	remaining: 15.9s
800:	learn: 0.6658411	total: 1m 3s	remaining: 15.8s
801:	learn: 0.6649786	total: 1m 3s	remaining: 15.7s
802:	learn: 0.6641930	total: 1m 3s	remaining: 15.6s
803:	learn: 0.6640212	total: 1m 3s	remaining: 15.5s
804:	learn: 0.66

944:	learn: 0.5508126	total: 1m 14s	remaining: 4.36s
945:	learn: 0.5499985	total: 1m 15s	remaining: 4.28s
946:	learn: 0.5498694	total: 1m 15s	remaining: 4.2s
947:	learn: 0.5483601	total: 1m 15s	remaining: 4.12s
948:	learn: 0.5479226	total: 1m 15s	remaining: 4.04s
949:	learn: 0.5476949	total: 1m 15s	remaining: 3.96s
950:	learn: 0.5462035	total: 1m 15s	remaining: 3.89s
951:	learn: 0.5451098	total: 1m 15s	remaining: 3.81s
952:	learn: 0.5447662	total: 1m 15s	remaining: 3.73s
953:	learn: 0.5443644	total: 1m 15s	remaining: 3.65s
954:	learn: 0.5436812	total: 1m 15s	remaining: 3.57s
955:	learn: 0.5432244	total: 1m 15s	remaining: 3.49s
956:	learn: 0.5428192	total: 1m 15s	remaining: 3.41s
957:	learn: 0.5424470	total: 1m 15s	remaining: 3.33s
958:	learn: 0.5411662	total: 1m 16s	remaining: 3.25s
959:	learn: 0.5404287	total: 1m 16s	remaining: 3.17s
960:	learn: 0.5395687	total: 1m 16s	remaining: 3.09s
961:	learn: 0.5386942	total: 1m 16s	remaining: 3.01s
962:	learn: 0.5381593	total: 1m 16s	remaining: 

In [9]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

1.4508498887927188
1.4542250739886105
1.44096181958973
1.487056248159563
1.428505485489778
1.4435008167359125
0.696903638022247
0.6850586247072452
0.6924405937333993
0.6797964562267367
0.7225403460886115
0.7046821693552419


In [10]:
train_size = 3000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)


Learning rate set to 0.048704
0:	learn: 4.9834280	total: 113ms	remaining: 1m 53s
1:	learn: 4.8702286	total: 207ms	remaining: 1m 43s
2:	learn: 4.7670693	total: 308ms	remaining: 1m 42s
3:	learn: 4.6797996	total: 402ms	remaining: 1m 40s
4:	learn: 4.5948133	total: 497ms	remaining: 1m 38s
5:	learn: 4.5174677	total: 591ms	remaining: 1m 37s
6:	learn: 4.4311955	total: 685ms	remaining: 1m 37s
7:	learn: 4.3633889	total: 779ms	remaining: 1m 36s
8:	learn: 4.2829047	total: 873ms	remaining: 1m 36s
9:	learn: 4.2066748	total: 968ms	remaining: 1m 35s
10:	learn: 4.1415765	total: 1.06s	remaining: 1m 35s
11:	learn: 4.0811226	total: 1.16s	remaining: 1m 35s
12:	learn: 4.0246059	total: 1.25s	remaining: 1m 35s
13:	learn: 3.9700833	total: 1.35s	remaining: 1m 35s
14:	learn: 3.9149739	total: 1.44s	remaining: 1m 34s
15:	learn: 3.8610567	total: 1.54s	remaining: 1m 34s
16:	learn: 3.8053840	total: 1.63s	remaining: 1m 34s
17:	learn: 3.7501733	total: 1.73s	remaining: 1m 34s
18:	learn: 3.6951303	total: 1.82s	remaining:

157:	learn: 2.2061904	total: 14.9s	remaining: 1m 19s
158:	learn: 2.1981273	total: 15s	remaining: 1m 19s
159:	learn: 2.1930720	total: 15.1s	remaining: 1m 19s
160:	learn: 2.1863743	total: 15.2s	remaining: 1m 19s
161:	learn: 2.1814454	total: 15.3s	remaining: 1m 19s
162:	learn: 2.1800678	total: 15.4s	remaining: 1m 18s
163:	learn: 2.1774229	total: 15.5s	remaining: 1m 18s
164:	learn: 2.1693233	total: 15.6s	remaining: 1m 18s
165:	learn: 2.1646076	total: 15.7s	remaining: 1m 18s
166:	learn: 2.1633808	total: 15.8s	remaining: 1m 18s
167:	learn: 2.1544438	total: 15.9s	remaining: 1m 18s
168:	learn: 2.1495812	total: 15.9s	remaining: 1m 18s
169:	learn: 2.1412899	total: 16s	remaining: 1m 18s
170:	learn: 2.1360213	total: 16.1s	remaining: 1m 18s
171:	learn: 2.1302160	total: 16.2s	remaining: 1m 18s
172:	learn: 2.1237586	total: 16.3s	remaining: 1m 18s
173:	learn: 2.1199311	total: 16.4s	remaining: 1m 17s
174:	learn: 2.1168409	total: 16.5s	remaining: 1m 17s
175:	learn: 2.1156558	total: 16.6s	remaining: 1m 1

316:	learn: 1.5607853	total: 29.9s	remaining: 1m 4s
317:	learn: 1.5590658	total: 30s	remaining: 1m 4s
318:	learn: 1.5584101	total: 30.1s	remaining: 1m 4s
319:	learn: 1.5577710	total: 30.2s	remaining: 1m 4s
320:	learn: 1.5534255	total: 30.3s	remaining: 1m 4s
321:	learn: 1.5528631	total: 30.4s	remaining: 1m 3s
322:	learn: 1.5522741	total: 30.5s	remaining: 1m 3s
323:	learn: 1.5514189	total: 30.6s	remaining: 1m 3s
324:	learn: 1.5505644	total: 30.7s	remaining: 1m 3s
325:	learn: 1.5499986	total: 30.7s	remaining: 1m 3s
326:	learn: 1.5494675	total: 30.8s	remaining: 1m 3s
327:	learn: 1.5465783	total: 30.9s	remaining: 1m 3s
328:	learn: 1.5456002	total: 31s	remaining: 1m 3s
329:	learn: 1.5414253	total: 31.1s	remaining: 1m 3s
330:	learn: 1.5400827	total: 31.2s	remaining: 1m 3s
331:	learn: 1.5395325	total: 31.3s	remaining: 1m 2s
332:	learn: 1.5377938	total: 31.4s	remaining: 1m 2s
333:	learn: 1.5342522	total: 31.5s	remaining: 1m 2s
334:	learn: 1.5333149	total: 31.6s	remaining: 1m 2s
335:	learn: 1.53

477:	learn: 1.2175987	total: 45.1s	remaining: 49.2s
478:	learn: 1.2151778	total: 45.2s	remaining: 49.1s
479:	learn: 1.2134887	total: 45.3s	remaining: 49s
480:	learn: 1.2113330	total: 45.4s	remaining: 48.9s
481:	learn: 1.2097935	total: 45.5s	remaining: 48.8s
482:	learn: 1.2073202	total: 45.5s	remaining: 48.8s
483:	learn: 1.2051064	total: 45.6s	remaining: 48.7s
484:	learn: 1.2029406	total: 45.7s	remaining: 48.6s
485:	learn: 1.2016657	total: 45.8s	remaining: 48.5s
486:	learn: 1.2000168	total: 45.9s	remaining: 48.4s
487:	learn: 1.1984771	total: 46s	remaining: 48.3s
488:	learn: 1.1963651	total: 46.1s	remaining: 48.2s
489:	learn: 1.1952662	total: 46.2s	remaining: 48.1s
490:	learn: 1.1937588	total: 46.3s	remaining: 48s
491:	learn: 1.1918884	total: 46.4s	remaining: 47.9s
492:	learn: 1.1902994	total: 46.5s	remaining: 47.8s
493:	learn: 1.1891214	total: 46.6s	remaining: 47.7s
494:	learn: 1.1882115	total: 46.7s	remaining: 47.6s
495:	learn: 1.1875458	total: 46.8s	remaining: 47.5s
496:	learn: 1.1862

638:	learn: 0.9891253	total: 1m	remaining: 34.1s
639:	learn: 0.9888755	total: 1m	remaining: 34s
640:	learn: 0.9886218	total: 1m	remaining: 33.9s
641:	learn: 0.9872788	total: 1m	remaining: 33.8s
642:	learn: 0.9849559	total: 1m	remaining: 33.7s
643:	learn: 0.9834212	total: 1m	remaining: 33.6s
644:	learn: 0.9821072	total: 1m	remaining: 33.5s
645:	learn: 0.9812090	total: 1m	remaining: 33.4s
646:	learn: 0.9791774	total: 1m 1s	remaining: 33.3s
647:	learn: 0.9778737	total: 1m 1s	remaining: 33.2s
648:	learn: 0.9769232	total: 1m 1s	remaining: 33.1s
649:	learn: 0.9766839	total: 1m 1s	remaining: 33s
650:	learn: 0.9748653	total: 1m 1s	remaining: 32.9s
651:	learn: 0.9739141	total: 1m 1s	remaining: 32.8s
652:	learn: 0.9725447	total: 1m 1s	remaining: 32.7s
653:	learn: 0.9704946	total: 1m 1s	remaining: 32.6s
654:	learn: 0.9692902	total: 1m 1s	remaining: 32.6s
655:	learn: 0.9678308	total: 1m 1s	remaining: 32.5s
656:	learn: 0.9669378	total: 1m 1s	remaining: 32.4s
657:	learn: 0.9646720	total: 1m 2s	remai

...

282:	learn: 1.6683183	total: 26.6s	remaining: 1m 7s
283:	learn: 1.6671229	total: 26.7s	remaining: 1m 7s
284:	learn: 1.6630633	total: 26.8s	remaining: 1m 7s
285:	learn: 1.6623633	total: 26.9s	remaining: 1m 7s
286:	learn: 1.6566323	total: 27s	remaining: 1m 7s
287:	learn: 1.6511218	total: 27.1s	remaining: 1m 7s
288:	learn: 1.6504406	total: 27.2s	remaining: 1m 6s
289:	learn: 1.6471159	total: 27.3s	remaining: 1m 6s
290:	learn: 1.6447309	total: 27.4s	remaining: 1m 6s
291:	learn: 1.6427356	total: 27.5s	remaining: 1m 6s
292:	learn: 1.6421118	total: 27.6s	remaining: 1m 6s
293:	learn: 1.6381955	total: 27.7s	remaining: 1m 6s
294:	learn: 1.6373337	total: 27.8s	remaining: 1m 6s
295:	learn: 1.6346232	total: 27.9s	remaining: 1m 6s
296:	learn: 1.6309802	total: 28s	remaining: 1m 6s
297:	learn: 1.6278431	total: 28.1s	remaining: 1m 6s
298:	learn: 1.6272197	total: 28.1s	remaining: 1m 5s
299:	learn: 1.6230659	total: 28.2s	remaining: 1m 5s
300:	learn: 1.6210933	total: 28.3s	remaining: 1m 5s
301:	learn: 1.62

442:	learn: 1.2983623	total: 41.7s	remaining: 52.4s
443:	learn: 1.2979506	total: 41.8s	remaining: 52.3s
444:	learn: 1.2972739	total: 41.9s	remaining: 52.2s
445:	learn: 1.2968477	total: 42s	remaining: 52.1s
446:	learn: 1.2964472	total: 42.1s	remaining: 52.1s
447:	learn: 1.2960594	total: 42.2s	remaining: 52s
448:	learn: 1.2931640	total: 42.3s	remaining: 51.9s
449:	learn: 1.2927680	total: 42.4s	remaining: 51.8s
450:	learn: 1.2923757	total: 42.4s	remaining: 51.7s
451:	learn: 1.2900679	total: 42.5s	remaining: 51.6s
452:	learn: 1.2888201	total: 42.6s	remaining: 51.5s
453:	learn: 1.2884631	total: 42.7s	remaining: 51.4s
454:	learn: 1.2879486	total: 42.8s	remaining: 51.3s
455:	learn: 1.2850120	total: 42.9s	remaining: 51.2s
456:	learn: 1.2828717	total: 43s	remaining: 51.1s
457:	learn: 1.2807496	total: 43.1s	remaining: 51s
458:	learn: 1.2791877	total: 43.2s	remaining: 50.9s
459:	learn: 1.2788043	total: 43.3s	remaining: 50.8s
460:	learn: 1.2770905	total: 43.4s	remaining: 50.7s
461:	learn: 1.274281

603:	learn: 1.0467580	total: 56.8s	remaining: 37.3s
604:	learn: 1.0452936	total: 56.9s	remaining: 37.2s
605:	learn: 1.0439966	total: 57s	remaining: 37.1s
606:	learn: 1.0435277	total: 57.1s	remaining: 37s
607:	learn: 1.0413755	total: 57.2s	remaining: 36.9s
608:	learn: 1.0408436	total: 57.3s	remaining: 36.8s
609:	learn: 1.0392432	total: 57.4s	remaining: 36.7s
610:	learn: 1.0386688	total: 57.5s	remaining: 36.6s
611:	learn: 1.0381514	total: 57.6s	remaining: 36.5s
612:	learn: 1.0374576	total: 57.7s	remaining: 36.4s
613:	learn: 1.0356385	total: 57.8s	remaining: 36.3s
614:	learn: 1.0340082	total: 57.9s	remaining: 36.2s
615:	learn: 1.0332835	total: 58s	remaining: 36.1s
616:	learn: 1.0315692	total: 58.1s	remaining: 36s
617:	learn: 1.0290802	total: 58.2s	remaining: 36s
618:	learn: 1.0285762	total: 58.3s	remaining: 35.9s
619:	learn: 1.0270676	total: 58.4s	remaining: 35.8s
620:	learn: 1.0259001	total: 58.4s	remaining: 35.7s
621:	learn: 1.0254154	total: 58.5s	remaining: 35.6s
622:	learn: 1.0238712	

762:	learn: 0.8672277	total: 1m 11s	remaining: 22.3s
763:	learn: 0.8658239	total: 1m 11s	remaining: 22.2s
764:	learn: 0.8654693	total: 1m 12s	remaining: 22.1s
765:	learn: 0.8643230	total: 1m 12s	remaining: 22s
766:	learn: 0.8637465	total: 1m 12s	remaining: 21.9s
767:	learn: 0.8627321	total: 1m 12s	remaining: 21.8s
768:	learn: 0.8613420	total: 1m 12s	remaining: 21.7s
769:	learn: 0.8608960	total: 1m 12s	remaining: 21.6s
770:	learn: 0.8607047	total: 1m 12s	remaining: 21.6s
771:	learn: 0.8600331	total: 1m 12s	remaining: 21.5s
772:	learn: 0.8590729	total: 1m 12s	remaining: 21.4s
773:	learn: 0.8574305	total: 1m 12s	remaining: 21.3s
774:	learn: 0.8564420	total: 1m 12s	remaining: 21.2s
775:	learn: 0.8559409	total: 1m 13s	remaining: 21.1s
776:	learn: 0.8549458	total: 1m 13s	remaining: 21s
777:	learn: 0.8541540	total: 1m 13s	remaining: 20.9s
778:	learn: 0.8524052	total: 1m 13s	remaining: 20.8s
779:	learn: 0.8514025	total: 1m 13s	remaining: 20.7s
780:	learn: 0.8510361	total: 1m 13s	remaining: 20.

918:	learn: 0.7314435	total: 1m 26s	remaining: 7.62s
919:	learn: 0.7304146	total: 1m 26s	remaining: 7.53s
920:	learn: 0.7301922	total: 1m 26s	remaining: 7.43s
921:	learn: 0.7298014	total: 1m 26s	remaining: 7.34s
922:	learn: 0.7292051	total: 1m 26s	remaining: 7.25s
923:	learn: 0.7283645	total: 1m 26s	remaining: 7.15s
924:	learn: 0.7279769	total: 1m 27s	remaining: 7.06s
925:	learn: 0.7268883	total: 1m 27s	remaining: 6.96s
926:	learn: 0.7262982	total: 1m 27s	remaining: 6.87s
927:	learn: 0.7258845	total: 1m 27s	remaining: 6.78s
928:	learn: 0.7243545	total: 1m 27s	remaining: 6.68s
929:	learn: 0.7239806	total: 1m 27s	remaining: 6.59s
930:	learn: 0.7230477	total: 1m 27s	remaining: 6.49s
931:	learn: 0.7225621	total: 1m 27s	remaining: 6.4s
932:	learn: 0.7224359	total: 1m 27s	remaining: 6.3s
933:	learn: 0.7217426	total: 1m 27s	remaining: 6.21s
934:	learn: 0.7210484	total: 1m 27s	remaining: 6.12s
935:	learn: 0.7200689	total: 1m 28s	remaining: 6.02s
936:	learn: 0.7189412	total: 1m 28s	remaining: 5

In [11]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

1.3924423826876657
1.4215350920414276
1.3858244212539905
1.3993040661358631
1.3781363435145801
1.3774119904924673
0.7207119482679873
0.7002679629109722
0.697758268903635
0.7338889594629384
0.7409522684230114
0.7306922816393795


In [12]:
train_size = 4000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)

Learning rate set to 0.050969
0:	learn: 5.1062042	total: 130ms	remaining: 2m 9s
1:	learn: 4.9997087	total: 236ms	remaining: 1m 57s
2:	learn: 4.8983536	total: 343ms	remaining: 1m 53s
3:	learn: 4.7868570	total: 450ms	remaining: 1m 52s
4:	learn: 4.6934830	total: 557ms	remaining: 1m 50s
5:	learn: 4.6091817	total: 664ms	remaining: 1m 50s
6:	learn: 4.5234949	total: 771ms	remaining: 1m 49s
7:	learn: 4.4484017	total: 877ms	remaining: 1m 48s
8:	learn: 4.3642779	total: 983ms	remaining: 1m 48s
9:	learn: 4.2905350	total: 1.09s	remaining: 1m 47s
10:	learn: 4.2180783	total: 1.2s	remaining: 1m 47s
11:	learn: 4.1503286	total: 1.3s	remaining: 1m 47s
12:	learn: 4.0872967	total: 1.41s	remaining: 1m 47s
13:	learn: 4.0229824	total: 1.52s	remaining: 1m 46s
14:	learn: 3.9602563	total: 1.62s	remaining: 1m 46s
15:	learn: 3.9106450	total: 1.73s	remaining: 1m 46s
16:	learn: 3.8607412	total: 1.84s	remaining: 1m 46s
17:	learn: 3.8134093	total: 1.94s	remaining: 1m 46s
18:	learn: 3.7689950	total: 2.05s	remaining: 1m

157:	learn: 2.2762426	total: 16.8s	remaining: 1m 29s
158:	learn: 2.2739616	total: 16.9s	remaining: 1m 29s
159:	learn: 2.2675944	total: 17s	remaining: 1m 29s
160:	learn: 2.2629376	total: 17.1s	remaining: 1m 28s
161:	learn: 2.2608070	total: 17.2s	remaining: 1m 28s
162:	learn: 2.2509327	total: 17.3s	remaining: 1m 28s
163:	learn: 2.2465858	total: 17.4s	remaining: 1m 28s
164:	learn: 2.2402231	total: 17.5s	remaining: 1m 28s
165:	learn: 2.2378960	total: 17.6s	remaining: 1m 28s
166:	learn: 2.2322462	total: 17.7s	remaining: 1m 28s
167:	learn: 2.2311383	total: 17.8s	remaining: 1m 28s
168:	learn: 2.2293189	total: 17.9s	remaining: 1m 28s
169:	learn: 2.2281050	total: 18s	remaining: 1m 27s
170:	learn: 2.2200902	total: 18.1s	remaining: 1m 27s
171:	learn: 2.2156129	total: 18.2s	remaining: 1m 27s
172:	learn: 2.2084412	total: 18.3s	remaining: 1m 27s
173:	learn: 2.2061661	total: 18.4s	remaining: 1m 27s
174:	learn: 2.2046144	total: 18.5s	remaining: 1m 27s
175:	learn: 2.2029895	total: 18.6s	remaining: 1m 2

313:	learn: 1.6777917	total: 33.3s	remaining: 1m 12s
314:	learn: 1.6734702	total: 33.4s	remaining: 1m 12s
315:	learn: 1.6710381	total: 33.5s	remaining: 1m 12s
316:	learn: 1.6671756	total: 33.6s	remaining: 1m 12s
317:	learn: 1.6644946	total: 33.7s	remaining: 1m 12s
318:	learn: 1.6614917	total: 33.8s	remaining: 1m 12s
319:	learn: 1.6587942	total: 33.9s	remaining: 1m 12s
320:	learn: 1.6548480	total: 34s	remaining: 1m 11s
321:	learn: 1.6536279	total: 34.1s	remaining: 1m 11s
322:	learn: 1.6530418	total: 34.2s	remaining: 1m 11s
323:	learn: 1.6496839	total: 34.3s	remaining: 1m 11s
324:	learn: 1.6463453	total: 34.4s	remaining: 1m 11s
325:	learn: 1.6419656	total: 34.5s	remaining: 1m 11s
326:	learn: 1.6410049	total: 34.7s	remaining: 1m 11s
327:	learn: 1.6391911	total: 34.8s	remaining: 1m 11s
328:	learn: 1.6382604	total: 34.9s	remaining: 1m 11s
329:	learn: 1.6357866	total: 35s	remaining: 1m 11s
330:	learn: 1.6343939	total: 35.1s	remaining: 1m 10s
331:	learn: 1.6310119	total: 35.2s	remaining: 1m 1

473:	learn: 1.3263239	total: 50.3s	remaining: 55.8s
474:	learn: 1.3248327	total: 50.4s	remaining: 55.7s
475:	learn: 1.3226392	total: 50.5s	remaining: 55.6s
476:	learn: 1.3212582	total: 50.6s	remaining: 55.5s
477:	learn: 1.3191552	total: 50.7s	remaining: 55.4s
478:	learn: 1.3162530	total: 50.8s	remaining: 55.3s
479:	learn: 1.3146825	total: 50.9s	remaining: 55.2s
480:	learn: 1.3141142	total: 51s	remaining: 55s
481:	learn: 1.3118832	total: 51.1s	remaining: 54.9s
482:	learn: 1.3115389	total: 51.2s	remaining: 54.8s
483:	learn: 1.3094624	total: 51.3s	remaining: 54.7s
484:	learn: 1.3080363	total: 51.4s	remaining: 54.6s
485:	learn: 1.3066785	total: 51.5s	remaining: 54.5s
486:	learn: 1.3043864	total: 51.7s	remaining: 54.4s
487:	learn: 1.3035816	total: 51.8s	remaining: 54.3s
488:	learn: 1.3015046	total: 51.9s	remaining: 54.2s
489:	learn: 1.3011834	total: 52s	remaining: 54.1s
490:	learn: 1.2997485	total: 52.1s	remaining: 54s
491:	learn: 1.2991674	total: 52.2s	remaining: 53.9s
492:	learn: 1.298532

633:	learn: 1.1076323	total: 1m 7s	remaining: 38.8s
634:	learn: 1.1051951	total: 1m 7s	remaining: 38.7s
635:	learn: 1.1034119	total: 1m 7s	remaining: 38.6s
636:	learn: 1.1029815	total: 1m 7s	remaining: 38.5s
637:	learn: 1.1013475	total: 1m 7s	remaining: 38.4s
638:	learn: 1.1000697	total: 1m 7s	remaining: 38.3s
639:	learn: 1.0989321	total: 1m 7s	remaining: 38.2s
640:	learn: 1.0976888	total: 1m 7s	remaining: 38.1s
641:	learn: 1.0970046	total: 1m 8s	remaining: 38s
642:	learn: 1.0957343	total: 1m 8s	remaining: 37.9s
643:	learn: 1.0951480	total: 1m 8s	remaining: 37.8s
644:	learn: 1.0938503	total: 1m 8s	remaining: 37.6s
645:	learn: 1.0924707	total: 1m 8s	remaining: 37.5s
646:	learn: 1.0919452	total: 1m 8s	remaining: 37.4s
647:	learn: 1.0907605	total: 1m 8s	remaining: 37.3s
648:	learn: 1.0887921	total: 1m 8s	remaining: 37.2s
649:	learn: 1.0875205	total: 1m 8s	remaining: 37.1s
650:	learn: 1.0861757	total: 1m 9s	remaining: 37s
651:	learn: 1.0851721	total: 1m 9s	remaining: 36.9s
652:	learn: 1.08

...

253:	learn: 1.8283360	total: 26.9s	remaining: 1m 19s
254:	learn: 1.8271455	total: 27s	remaining: 1m 18s
255:	learn: 1.8242184	total: 27.1s	remaining: 1m 18s
256:	learn: 1.8213043	total: 27.2s	remaining: 1m 18s
257:	learn: 1.8206354	total: 27.3s	remaining: 1m 18s
258:	learn: 1.8158182	total: 27.4s	remaining: 1m 18s
259:	learn: 1.8142680	total: 27.5s	remaining: 1m 18s
260:	learn: 1.8108127	total: 27.7s	remaining: 1m 18s
261:	learn: 1.8070069	total: 27.8s	remaining: 1m 18s
262:	learn: 1.8039861	total: 27.9s	remaining: 1m 18s
263:	learn: 1.8030974	total: 28s	remaining: 1m 17s
264:	learn: 1.8024546	total: 28.1s	remaining: 1m 17s
265:	learn: 1.7983419	total: 28.2s	remaining: 1m 17s
266:	learn: 1.7960122	total: 28.3s	remaining: 1m 17s
267:	learn: 1.7919424	total: 28.4s	remaining: 1m 17s
268:	learn: 1.7877269	total: 28.5s	remaining: 1m 17s
269:	learn: 1.7825941	total: 28.6s	remaining: 1m 17s
270:	learn: 1.7795859	total: 28.7s	remaining: 1m 17s
271:	learn: 1.7766356	total: 28.8s	remaining: 1m 1

411:	learn: 1.4487633	total: 43.6s	remaining: 1m 2s
412:	learn: 1.4468896	total: 43.7s	remaining: 1m 2s
413:	learn: 1.4444139	total: 43.8s	remaining: 1m 2s
414:	learn: 1.4427752	total: 43.9s	remaining: 1m 1s
415:	learn: 1.4401190	total: 44s	remaining: 1m 1s
416:	learn: 1.4390528	total: 44.1s	remaining: 1m 1s
417:	learn: 1.4373139	total: 44.2s	remaining: 1m 1s
418:	learn: 1.4369056	total: 44.4s	remaining: 1m 1s
419:	learn: 1.4359293	total: 44.5s	remaining: 1m 1s
420:	learn: 1.4339747	total: 44.6s	remaining: 1m 1s
421:	learn: 1.4315677	total: 44.7s	remaining: 1m 1s
422:	learn: 1.4311986	total: 44.8s	remaining: 1m 1s
423:	learn: 1.4284034	total: 44.9s	remaining: 1m
424:	learn: 1.4261628	total: 45s	remaining: 1m
425:	learn: 1.4257863	total: 45.1s	remaining: 1m
426:	learn: 1.4232101	total: 45.2s	remaining: 1m
427:	learn: 1.4209019	total: 45.3s	remaining: 1m
428:	learn: 1.4187560	total: 45.4s	remaining: 1m
429:	learn: 1.4162580	total: 45.5s	remaining: 1m
430:	learn: 1.4145373	total: 45.6s	re

571:	learn: 1.1985343	total: 1m	remaining: 45.3s
572:	learn: 1.1974860	total: 1m	remaining: 45.2s
573:	learn: 1.1958225	total: 1m	remaining: 45.1s
574:	learn: 1.1947516	total: 1m	remaining: 45s
575:	learn: 1.1938734	total: 1m	remaining: 44.9s
576:	learn: 1.1930245	total: 1m 1s	remaining: 44.8s
577:	learn: 1.1919192	total: 1m 1s	remaining: 44.7s
578:	learn: 1.1901864	total: 1m 1s	remaining: 44.6s
579:	learn: 1.1887472	total: 1m 1s	remaining: 44.4s
580:	learn: 1.1861510	total: 1m 1s	remaining: 44.3s
581:	learn: 1.1845089	total: 1m 1s	remaining: 44.2s
582:	learn: 1.1832022	total: 1m 1s	remaining: 44.1s
583:	learn: 1.1825342	total: 1m 1s	remaining: 44s
584:	learn: 1.1806973	total: 1m 1s	remaining: 43.9s
585:	learn: 1.1790576	total: 1m 2s	remaining: 43.8s
586:	learn: 1.1788024	total: 1m 2s	remaining: 43.7s
587:	learn: 1.1773358	total: 1m 2s	remaining: 43.6s
588:	learn: 1.1741208	total: 1m 2s	remaining: 43.5s
589:	learn: 1.1734044	total: 1m 2s	remaining: 43.4s
590:	learn: 1.1725154	total: 1m

729:	learn: 1.0163614	total: 1m 17s	remaining: 28.6s
730:	learn: 1.0160021	total: 1m 17s	remaining: 28.5s
731:	learn: 1.0142864	total: 1m 17s	remaining: 28.4s
732:	learn: 1.0135715	total: 1m 17s	remaining: 28.3s
733:	learn: 1.0120472	total: 1m 17s	remaining: 28.1s
734:	learn: 1.0107599	total: 1m 17s	remaining: 28s
735:	learn: 1.0103211	total: 1m 17s	remaining: 27.9s
736:	learn: 1.0092312	total: 1m 17s	remaining: 27.8s
737:	learn: 1.0085177	total: 1m 18s	remaining: 27.7s
738:	learn: 1.0080737	total: 1m 18s	remaining: 27.6s
739:	learn: 1.0071014	total: 1m 18s	remaining: 27.5s
740:	learn: 1.0067699	total: 1m 18s	remaining: 27.4s
741:	learn: 1.0054958	total: 1m 18s	remaining: 27.3s
742:	learn: 1.0037671	total: 1m 18s	remaining: 27.2s
743:	learn: 1.0021475	total: 1m 18s	remaining: 27.1s
744:	learn: 1.0019566	total: 1m 18s	remaining: 27s
745:	learn: 1.0012704	total: 1m 18s	remaining: 26.9s
746:	learn: 1.0001312	total: 1m 19s	remaining: 26.8s
747:	learn: 0.9987470	total: 1m 19s	remaining: 26.

885:	learn: 0.8762889	total: 1m 33s	remaining: 12.1s
886:	learn: 0.8747739	total: 1m 33s	remaining: 12s
887:	learn: 0.8745610	total: 1m 33s	remaining: 11.9s
888:	learn: 0.8737204	total: 1m 34s	remaining: 11.7s
889:	learn: 0.8724632	total: 1m 34s	remaining: 11.6s
890:	learn: 0.8720162	total: 1m 34s	remaining: 11.5s
891:	learn: 0.8707698	total: 1m 34s	remaining: 11.4s
892:	learn: 0.8704729	total: 1m 34s	remaining: 11.3s
893:	learn: 0.8703013	total: 1m 34s	remaining: 11.2s
894:	learn: 0.8696792	total: 1m 34s	remaining: 11.1s
895:	learn: 0.8688218	total: 1m 34s	remaining: 11s
896:	learn: 0.8686218	total: 1m 34s	remaining: 10.9s
897:	learn: 0.8678438	total: 1m 35s	remaining: 10.8s
898:	learn: 0.8666595	total: 1m 35s	remaining: 10.7s
899:	learn: 0.8656784	total: 1m 35s	remaining: 10.6s
900:	learn: 0.8652016	total: 1m 35s	remaining: 10.5s
901:	learn: 0.8639161	total: 1m 35s	remaining: 10.4s
902:	learn: 0.8624273	total: 1m 35s	remaining: 10.3s
903:	learn: 0.8612469	total: 1m 35s	remaining: 10.

In [13]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

1.3522410012556674
1.3434533727796016
1.3583780564685397
1.3559294894343519
1.326944457036497
1.3764996305593458
0.745360572467871
0.7425101730367689
0.7364819424989113
0.7512324293092013
0.7689842496916399
0.7275940678028339


In [14]:
train_size = 5000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)

Learning rate set to 0.052798
0:	learn: 5.0965354	total: 144ms	remaining: 2m 23s
1:	learn: 4.9774275	total: 259ms	remaining: 2m 9s
2:	learn: 4.8674486	total: 376ms	remaining: 2m 4s
3:	learn: 4.7609044	total: 491ms	remaining: 2m 2s
4:	learn: 4.6594221	total: 607ms	remaining: 2m
5:	learn: 4.5645479	total: 724ms	remaining: 1m 59s
6:	learn: 4.4685211	total: 841ms	remaining: 1m 59s
7:	learn: 4.3777422	total: 959ms	remaining: 1m 58s
8:	learn: 4.2982956	total: 1.07s	remaining: 1m 58s
9:	learn: 4.2271946	total: 1.19s	remaining: 1m 57s
10:	learn: 4.1504414	total: 1.31s	remaining: 1m 57s
11:	learn: 4.0818140	total: 1.42s	remaining: 1m 57s
12:	learn: 4.0180376	total: 1.54s	remaining: 1m 56s
13:	learn: 3.9585455	total: 1.66s	remaining: 1m 56s
14:	learn: 3.8949780	total: 1.77s	remaining: 1m 56s
15:	learn: 3.8392040	total: 1.89s	remaining: 1m 56s
16:	learn: 3.7881242	total: 2s	remaining: 1m 55s
17:	learn: 3.7375072	total: 2.12s	remaining: 1m 55s
18:	learn: 3.6914635	total: 2.24s	remaining: 1m 55s
19

157:	learn: 2.2584465	total: 18.3s	remaining: 1m 37s
158:	learn: 2.2567243	total: 18.4s	remaining: 1m 37s
159:	learn: 2.2547614	total: 18.5s	remaining: 1m 37s
160:	learn: 2.2496203	total: 18.6s	remaining: 1m 37s
161:	learn: 2.2448137	total: 18.7s	remaining: 1m 36s
162:	learn: 2.2418174	total: 18.9s	remaining: 1m 36s
163:	learn: 2.2356279	total: 19s	remaining: 1m 36s
164:	learn: 2.2282818	total: 19.1s	remaining: 1m 36s
165:	learn: 2.2269847	total: 19.2s	remaining: 1m 36s
166:	learn: 2.2214752	total: 19.3s	remaining: 1m 36s
167:	learn: 2.2199449	total: 19.4s	remaining: 1m 36s
168:	learn: 2.2135592	total: 19.6s	remaining: 1m 36s
169:	learn: 2.2118407	total: 19.7s	remaining: 1m 36s
170:	learn: 2.2105347	total: 19.8s	remaining: 1m 35s
171:	learn: 2.2076202	total: 19.9s	remaining: 1m 35s
172:	learn: 2.2012894	total: 20s	remaining: 1m 35s
173:	learn: 2.1962901	total: 20.1s	remaining: 1m 35s
174:	learn: 2.1911157	total: 20.2s	remaining: 1m 35s
175:	learn: 2.1874546	total: 20.4s	remaining: 1m 3

313:	learn: 1.7220704	total: 36.3s	remaining: 1m 19s
314:	learn: 1.7191187	total: 36.4s	remaining: 1m 19s
315:	learn: 1.7171007	total: 36.5s	remaining: 1m 18s
316:	learn: 1.7131214	total: 36.6s	remaining: 1m 18s
317:	learn: 1.7114842	total: 36.7s	remaining: 1m 18s
318:	learn: 1.7109977	total: 36.8s	remaining: 1m 18s
319:	learn: 1.7076623	total: 37s	remaining: 1m 18s
320:	learn: 1.7064034	total: 37.1s	remaining: 1m 18s
321:	learn: 1.7053909	total: 37.2s	remaining: 1m 18s
322:	learn: 1.7020226	total: 37.3s	remaining: 1m 18s
323:	learn: 1.7000729	total: 37.4s	remaining: 1m 18s
324:	learn: 1.6968420	total: 37.5s	remaining: 1m 17s
325:	learn: 1.6961496	total: 37.6s	remaining: 1m 17s
326:	learn: 1.6931809	total: 37.8s	remaining: 1m 17s
327:	learn: 1.6901330	total: 37.9s	remaining: 1m 17s
328:	learn: 1.6884807	total: 38s	remaining: 1m 17s
329:	learn: 1.6874800	total: 38.1s	remaining: 1m 17s
330:	learn: 1.6846395	total: 38.2s	remaining: 1m 17s
331:	learn: 1.6838665	total: 38.3s	remaining: 1m 1

471:	learn: 1.3950321	total: 54.5s	remaining: 1m
472:	learn: 1.3939703	total: 54.6s	remaining: 1m
473:	learn: 1.3925590	total: 54.7s	remaining: 1m
474:	learn: 1.3907443	total: 54.9s	remaining: 1m
475:	learn: 1.3886966	total: 55s	remaining: 1m
476:	learn: 1.3884032	total: 55.1s	remaining: 1m
477:	learn: 1.3857328	total: 55.2s	remaining: 1m
478:	learn: 1.3837975	total: 55.3s	remaining: 1m
479:	learn: 1.3830534	total: 55.4s	remaining: 1m
480:	learn: 1.3809359	total: 55.5s	remaining: 59.9s
481:	learn: 1.3791162	total: 55.7s	remaining: 59.8s
482:	learn: 1.3772847	total: 55.8s	remaining: 59.7s
483:	learn: 1.3750531	total: 55.9s	remaining: 59.6s
484:	learn: 1.3736738	total: 56s	remaining: 59.5s
485:	learn: 1.3715147	total: 56.1s	remaining: 59.4s
486:	learn: 1.3699455	total: 56.2s	remaining: 59.2s
487:	learn: 1.3680805	total: 56.4s	remaining: 59.1s
488:	learn: 1.3673987	total: 56.5s	remaining: 59s
489:	learn: 1.3647673	total: 56.6s	remaining: 58.9s
490:	learn: 1.3636158	total: 56.7s	remaining:

631:	learn: 1.1779814	total: 1m 12s	remaining: 42.5s
632:	learn: 1.1767145	total: 1m 13s	remaining: 42.4s
633:	learn: 1.1753563	total: 1m 13s	remaining: 42.3s
634:	learn: 1.1724846	total: 1m 13s	remaining: 42.2s
635:	learn: 1.1707512	total: 1m 13s	remaining: 42s
636:	learn: 1.1699982	total: 1m 13s	remaining: 41.9s
637:	learn: 1.1687584	total: 1m 13s	remaining: 41.8s
638:	learn: 1.1682723	total: 1m 13s	remaining: 41.7s
639:	learn: 1.1668463	total: 1m 13s	remaining: 41.6s
640:	learn: 1.1655164	total: 1m 14s	remaining: 41.5s
641:	learn: 1.1650460	total: 1m 14s	remaining: 41.3s
642:	learn: 1.1634783	total: 1m 14s	remaining: 41.2s
643:	learn: 1.1624368	total: 1m 14s	remaining: 41.1s
644:	learn: 1.1605906	total: 1m 14s	remaining: 41s
645:	learn: 1.1592896	total: 1m 14s	remaining: 40.9s
646:	learn: 1.1577829	total: 1m 14s	remaining: 40.8s
647:	learn: 1.1567268	total: 1m 14s	remaining: 40.7s
648:	learn: 1.1562471	total: 1m 14s	remaining: 40.5s
649:	learn: 1.1553022	total: 1m 15s	remaining: 40.

...

243:	learn: 1.9511447	total: 28s	remaining: 1m 26s
244:	learn: 1.9480574	total: 28.1s	remaining: 1m 26s
245:	learn: 1.9434469	total: 28.2s	remaining: 1m 26s
246:	learn: 1.9391809	total: 28.4s	remaining: 1m 26s
247:	learn: 1.9374149	total: 28.5s	remaining: 1m 26s
248:	learn: 1.9330693	total: 28.6s	remaining: 1m 26s
249:	learn: 1.9322577	total: 28.7s	remaining: 1m 26s
250:	learn: 1.9285936	total: 28.8s	remaining: 1m 25s
251:	learn: 1.9250202	total: 28.9s	remaining: 1m 25s
252:	learn: 1.9205580	total: 29s	remaining: 1m 25s
253:	learn: 1.9155369	total: 29.2s	remaining: 1m 25s
254:	learn: 1.9085270	total: 29.3s	remaining: 1m 25s
255:	learn: 1.9049409	total: 29.4s	remaining: 1m 25s
256:	learn: 1.9038803	total: 29.5s	remaining: 1m 25s
257:	learn: 1.9009112	total: 29.6s	remaining: 1m 25s
258:	learn: 1.8976760	total: 29.7s	remaining: 1m 25s
259:	learn: 1.8938132	total: 29.8s	remaining: 1m 24s
260:	learn: 1.8930008	total: 30s	remaining: 1m 24s
261:	learn: 1.8877361	total: 30.1s	remaining: 1m 24s

399:	learn: 1.5635292	total: 45.9s	remaining: 1m 8s
400:	learn: 1.5615418	total: 46s	remaining: 1m 8s
401:	learn: 1.5587878	total: 46.1s	remaining: 1m 8s
402:	learn: 1.5567212	total: 46.2s	remaining: 1m 8s
403:	learn: 1.5555050	total: 46.4s	remaining: 1m 8s
404:	learn: 1.5534270	total: 46.5s	remaining: 1m 8s
405:	learn: 1.5521603	total: 46.6s	remaining: 1m 8s
406:	learn: 1.5500485	total: 46.7s	remaining: 1m 8s
407:	learn: 1.5495484	total: 46.8s	remaining: 1m 7s
408:	learn: 1.5488078	total: 46.9s	remaining: 1m 7s
409:	learn: 1.5458108	total: 47s	remaining: 1m 7s
410:	learn: 1.5440810	total: 47.2s	remaining: 1m 7s
411:	learn: 1.5420871	total: 47.3s	remaining: 1m 7s
412:	learn: 1.5390262	total: 47.4s	remaining: 1m 7s
413:	learn: 1.5386111	total: 47.5s	remaining: 1m 7s
414:	learn: 1.5369213	total: 47.6s	remaining: 1m 7s
415:	learn: 1.5353345	total: 47.7s	remaining: 1m 7s
416:	learn: 1.5328467	total: 47.9s	remaining: 1m 6s
417:	learn: 1.5307702	total: 48s	remaining: 1m 6s
418:	learn: 1.5295

559:	learn: 1.3084544	total: 1m 4s	remaining: 50.5s
560:	learn: 1.3081532	total: 1m 4s	remaining: 50.4s
561:	learn: 1.3066412	total: 1m 4s	remaining: 50.2s
562:	learn: 1.3058650	total: 1m 4s	remaining: 50.1s
563:	learn: 1.3044160	total: 1m 4s	remaining: 50s
564:	learn: 1.3027637	total: 1m 4s	remaining: 49.9s
565:	learn: 1.3020733	total: 1m 4s	remaining: 49.8s
566:	learn: 1.3010910	total: 1m 5s	remaining: 49.7s
567:	learn: 1.3004536	total: 1m 5s	remaining: 49.6s
568:	learn: 1.3000350	total: 1m 5s	remaining: 49.4s
569:	learn: 1.2974913	total: 1m 5s	remaining: 49.3s
570:	learn: 1.2959407	total: 1m 5s	remaining: 49.2s
571:	learn: 1.2945847	total: 1m 5s	remaining: 49.1s
572:	learn: 1.2938667	total: 1m 5s	remaining: 49s
573:	learn: 1.2926984	total: 1m 5s	remaining: 48.9s
574:	learn: 1.2916333	total: 1m 5s	remaining: 48.8s
575:	learn: 1.2902077	total: 1m 6s	remaining: 48.6s
576:	learn: 1.2888569	total: 1m 6s	remaining: 48.5s
577:	learn: 1.2878460	total: 1m 6s	remaining: 48.4s
578:	learn: 1.28

717:	learn: 1.1266173	total: 1m 22s	remaining: 32.3s
718:	learn: 1.1252006	total: 1m 22s	remaining: 32.2s
719:	learn: 1.1237664	total: 1m 22s	remaining: 32.1s
720:	learn: 1.1226782	total: 1m 22s	remaining: 32s
721:	learn: 1.1214838	total: 1m 22s	remaining: 31.9s
722:	learn: 1.1199914	total: 1m 22s	remaining: 31.8s
723:	learn: 1.1182522	total: 1m 23s	remaining: 31.7s
724:	learn: 1.1168995	total: 1m 23s	remaining: 31.5s
725:	learn: 1.1163195	total: 1m 23s	remaining: 31.4s
726:	learn: 1.1157729	total: 1m 23s	remaining: 31.3s
727:	learn: 1.1152547	total: 1m 23s	remaining: 31.2s
728:	learn: 1.1141285	total: 1m 23s	remaining: 31.1s
729:	learn: 1.1122538	total: 1m 23s	remaining: 31s
730:	learn: 1.1102496	total: 1m 23s	remaining: 30.9s
731:	learn: 1.1094044	total: 1m 23s	remaining: 30.7s
732:	learn: 1.1080908	total: 1m 24s	remaining: 30.6s
733:	learn: 1.1068977	total: 1m 24s	remaining: 30.5s
734:	learn: 1.1066552	total: 1m 24s	remaining: 30.4s
735:	learn: 1.1053429	total: 1m 24s	remaining: 30.

873:	learn: 0.9809668	total: 1m 40s	remaining: 14.5s
874:	learn: 0.9805305	total: 1m 40s	remaining: 14.3s
875:	learn: 0.9797311	total: 1m 40s	remaining: 14.2s
876:	learn: 0.9783033	total: 1m 40s	remaining: 14.1s
877:	learn: 0.9773821	total: 1m 40s	remaining: 14s
878:	learn: 0.9769927	total: 1m 40s	remaining: 13.9s
879:	learn: 0.9758123	total: 1m 40s	remaining: 13.8s
880:	learn: 0.9750138	total: 1m 41s	remaining: 13.6s
881:	learn: 0.9746332	total: 1m 41s	remaining: 13.5s
882:	learn: 0.9742336	total: 1m 41s	remaining: 13.4s
883:	learn: 0.9729855	total: 1m 41s	remaining: 13.3s
884:	learn: 0.9721512	total: 1m 41s	remaining: 13.2s
885:	learn: 0.9720051	total: 1m 41s	remaining: 13.1s
886:	learn: 0.9705768	total: 1m 41s	remaining: 13s
887:	learn: 0.9697026	total: 1m 41s	remaining: 12.8s
888:	learn: 0.9683190	total: 1m 41s	remaining: 12.7s
889:	learn: 0.9672086	total: 1m 42s	remaining: 12.6s
890:	learn: 0.9670536	total: 1m 42s	remaining: 12.5s
891:	learn: 0.9656337	total: 1m 42s	remaining: 12.

In [15]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

1.3089518274075784
1.3039467222100511
1.3316575807466478
1.3040720888625517
1.2978790709358057
1.3072036742828363
0.7646399235760493
0.7429840582542857
0.747714339500656
0.7627505529258152
0.7847542073514746
0.7849964598480146


In [16]:
train_size = 6000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)

Learning rate set to 0.054341
0:	learn: 5.1191120	total: 147ms	remaining: 2m 26s
1:	learn: 5.0021161	total: 271ms	remaining: 2m 15s
2:	learn: 4.8781716	total: 395ms	remaining: 2m 11s
3:	learn: 4.7759104	total: 518ms	remaining: 2m 9s
4:	learn: 4.6761228	total: 641ms	remaining: 2m 7s
5:	learn: 4.5822570	total: 763ms	remaining: 2m 6s
6:	learn: 4.4862121	total: 887ms	remaining: 2m 5s
7:	learn: 4.4026715	total: 1.01s	remaining: 2m 5s
8:	learn: 4.3149302	total: 1.13s	remaining: 2m 4s
9:	learn: 4.2320699	total: 1.26s	remaining: 2m 4s
10:	learn: 4.1581112	total: 1.38s	remaining: 2m 4s
11:	learn: 4.0872742	total: 1.5s	remaining: 2m 3s
12:	learn: 4.0165471	total: 1.62s	remaining: 2m 3s
13:	learn: 3.9531414	total: 1.75s	remaining: 2m 3s
14:	learn: 3.8929563	total: 1.87s	remaining: 2m 2s
15:	learn: 3.8377245	total: 2s	remaining: 2m 2s
16:	learn: 3.7853848	total: 2.12s	remaining: 2m 2s
17:	learn: 3.7334336	total: 2.24s	remaining: 2m 2s
18:	learn: 3.6835064	total: 2.36s	remaining: 2m 2s
19:	learn: 3

159:	learn: 2.2584625	total: 19.5s	remaining: 1m 42s
160:	learn: 2.2554832	total: 19.6s	remaining: 1m 42s
161:	learn: 2.2497787	total: 19.8s	remaining: 1m 42s
162:	learn: 2.2454918	total: 19.9s	remaining: 1m 42s
163:	learn: 2.2390259	total: 20s	remaining: 1m 41s
164:	learn: 2.2309668	total: 20.1s	remaining: 1m 41s
165:	learn: 2.2268796	total: 20.3s	remaining: 1m 41s
166:	learn: 2.2258958	total: 20.4s	remaining: 1m 41s
167:	learn: 2.2215703	total: 20.5s	remaining: 1m 41s
168:	learn: 2.2170535	total: 20.6s	remaining: 1m 41s
169:	learn: 2.2151406	total: 20.7s	remaining: 1m 41s
170:	learn: 2.2122480	total: 20.9s	remaining: 1m 41s
171:	learn: 2.2108335	total: 21s	remaining: 1m 40s
172:	learn: 2.2098609	total: 21.1s	remaining: 1m 40s
173:	learn: 2.2038050	total: 21.2s	remaining: 1m 40s
174:	learn: 2.1988666	total: 21.3s	remaining: 1m 40s
175:	learn: 2.1940339	total: 21.5s	remaining: 1m 40s
176:	learn: 2.1854255	total: 21.6s	remaining: 1m 40s
177:	learn: 2.1814910	total: 21.7s	remaining: 1m 4

315:	learn: 1.7359958	total: 38.5s	remaining: 1m 23s
316:	learn: 1.7335412	total: 38.6s	remaining: 1m 23s
317:	learn: 1.7295471	total: 38.8s	remaining: 1m 23s
318:	learn: 1.7291108	total: 38.9s	remaining: 1m 22s
319:	learn: 1.7256503	total: 39s	remaining: 1m 22s
320:	learn: 1.7232572	total: 39.1s	remaining: 1m 22s
321:	learn: 1.7205913	total: 39.2s	remaining: 1m 22s
322:	learn: 1.7198917	total: 39.4s	remaining: 1m 22s
323:	learn: 1.7156361	total: 39.5s	remaining: 1m 22s
324:	learn: 1.7127323	total: 39.6s	remaining: 1m 22s
325:	learn: 1.7105130	total: 39.7s	remaining: 1m 22s
326:	learn: 1.7082956	total: 39.9s	remaining: 1m 22s
327:	learn: 1.7060018	total: 40s	remaining: 1m 21s
328:	learn: 1.7027266	total: 40.1s	remaining: 1m 21s
329:	learn: 1.7002364	total: 40.2s	remaining: 1m 21s
330:	learn: 1.6997820	total: 40.3s	remaining: 1m 21s
331:	learn: 1.6972469	total: 40.5s	remaining: 1m 21s
332:	learn: 1.6932515	total: 40.6s	remaining: 1m 21s
333:	learn: 1.6907406	total: 40.7s	remaining: 1m 2

473:	learn: 1.4292910	total: 57.8s	remaining: 1m 4s
474:	learn: 1.4266296	total: 57.9s	remaining: 1m 3s
475:	learn: 1.4252150	total: 58s	remaining: 1m 3s
476:	learn: 1.4243148	total: 58.1s	remaining: 1m 3s
477:	learn: 1.4220338	total: 58.2s	remaining: 1m 3s
478:	learn: 1.4209026	total: 58.4s	remaining: 1m 3s
479:	learn: 1.4188982	total: 58.5s	remaining: 1m 3s
480:	learn: 1.4179922	total: 58.6s	remaining: 1m 3s
481:	learn: 1.4156423	total: 58.7s	remaining: 1m 3s
482:	learn: 1.4143080	total: 58.9s	remaining: 1m 3s
483:	learn: 1.4122646	total: 59s	remaining: 1m 2s
484:	learn: 1.4107206	total: 59.1s	remaining: 1m 2s
485:	learn: 1.4098684	total: 59.2s	remaining: 1m 2s
486:	learn: 1.4084605	total: 59.3s	remaining: 1m 2s
487:	learn: 1.4067482	total: 59.5s	remaining: 1m 2s
488:	learn: 1.4043919	total: 59.6s	remaining: 1m 2s
489:	learn: 1.4026770	total: 59.7s	remaining: 1m 2s
490:	learn: 1.4009775	total: 59.8s	remaining: 1m 2s
491:	learn: 1.3998797	total: 60s	remaining: 1m 1s
492:	learn: 1.3982

633:	learn: 1.2026945	total: 1m 17s	remaining: 44.6s
634:	learn: 1.2016761	total: 1m 17s	remaining: 44.5s
635:	learn: 1.2007192	total: 1m 17s	remaining: 44.4s
636:	learn: 1.1993998	total: 1m 17s	remaining: 44.3s
637:	learn: 1.1979431	total: 1m 17s	remaining: 44.1s
638:	learn: 1.1976095	total: 1m 17s	remaining: 44s
639:	learn: 1.1960663	total: 1m 18s	remaining: 43.9s
640:	learn: 1.1950009	total: 1m 18s	remaining: 43.8s
641:	learn: 1.1937615	total: 1m 18s	remaining: 43.6s
642:	learn: 1.1925255	total: 1m 18s	remaining: 43.5s
643:	learn: 1.1915746	total: 1m 18s	remaining: 43.4s
644:	learn: 1.1906015	total: 1m 18s	remaining: 43.3s
645:	learn: 1.1892771	total: 1m 18s	remaining: 43.2s
646:	learn: 1.1874261	total: 1m 18s	remaining: 43s
647:	learn: 1.1865471	total: 1m 19s	remaining: 42.9s
648:	learn: 1.1854285	total: 1m 19s	remaining: 42.8s
649:	learn: 1.1841433	total: 1m 19s	remaining: 42.7s
650:	learn: 1.1833433	total: 1m 19s	remaining: 42.6s
651:	learn: 1.1817838	total: 1m 19s	remaining: 42.

...

247:	learn: 1.9320068	total: 30.1s	remaining: 1m 31s
248:	learn: 1.9286575	total: 30.2s	remaining: 1m 31s
249:	learn: 1.9244127	total: 30.4s	remaining: 1m 31s
250:	learn: 1.9236279	total: 30.5s	remaining: 1m 30s
251:	learn: 1.9218401	total: 30.6s	remaining: 1m 30s
252:	learn: 1.9171636	total: 30.7s	remaining: 1m 30s
253:	learn: 1.9150076	total: 30.8s	remaining: 1m 30s
254:	learn: 1.9122317	total: 31s	remaining: 1m 30s
255:	learn: 1.9068546	total: 31.1s	remaining: 1m 30s
256:	learn: 1.9039502	total: 31.2s	remaining: 1m 30s
257:	learn: 1.9010454	total: 31.3s	remaining: 1m 30s
258:	learn: 1.8976467	total: 31.4s	remaining: 1m 29s
259:	learn: 1.8919473	total: 31.6s	remaining: 1m 29s
260:	learn: 1.8887524	total: 31.7s	remaining: 1m 29s
261:	learn: 1.8880055	total: 31.8s	remaining: 1m 29s
262:	learn: 1.8833309	total: 31.9s	remaining: 1m 29s
263:	learn: 1.8810290	total: 32s	remaining: 1m 29s
264:	learn: 1.8787666	total: 32.2s	remaining: 1m 29s
265:	learn: 1.8751103	total: 32.3s	remaining: 1m 2

403:	learn: 1.5641566	total: 49s	remaining: 1m 12s
404:	learn: 1.5619188	total: 49.1s	remaining: 1m 12s
405:	learn: 1.5583224	total: 49.3s	remaining: 1m 12s
406:	learn: 1.5578138	total: 49.4s	remaining: 1m 11s
407:	learn: 1.5549886	total: 49.5s	remaining: 1m 11s
408:	learn: 1.5522102	total: 49.6s	remaining: 1m 11s
409:	learn: 1.5501942	total: 49.8s	remaining: 1m 11s
410:	learn: 1.5476083	total: 49.9s	remaining: 1m 11s
411:	learn: 1.5455290	total: 50s	remaining: 1m 11s
412:	learn: 1.5434604	total: 50.1s	remaining: 1m 11s
413:	learn: 1.5419178	total: 50.2s	remaining: 1m 11s
414:	learn: 1.5411965	total: 50.4s	remaining: 1m 10s
415:	learn: 1.5397463	total: 50.5s	remaining: 1m 10s
416:	learn: 1.5383930	total: 50.6s	remaining: 1m 10s
417:	learn: 1.5378612	total: 50.7s	remaining: 1m 10s
418:	learn: 1.5359907	total: 50.8s	remaining: 1m 10s
419:	learn: 1.5351872	total: 51s	remaining: 1m 10s
420:	learn: 1.5330439	total: 51.1s	remaining: 1m 10s
421:	learn: 1.5316135	total: 51.2s	remaining: 1m 10s

563:	learn: 1.3151617	total: 1m 8s	remaining: 52.9s
564:	learn: 1.3146654	total: 1m 8s	remaining: 52.8s
565:	learn: 1.3142990	total: 1m 8s	remaining: 52.7s
566:	learn: 1.3129338	total: 1m 8s	remaining: 52.5s
567:	learn: 1.3109589	total: 1m 8s	remaining: 52.4s
568:	learn: 1.3106062	total: 1m 9s	remaining: 52.3s
569:	learn: 1.3095352	total: 1m 9s	remaining: 52.2s
570:	learn: 1.3080379	total: 1m 9s	remaining: 52s
571:	learn: 1.3073187	total: 1m 9s	remaining: 51.9s
572:	learn: 1.3066491	total: 1m 9s	remaining: 51.8s
573:	learn: 1.3055116	total: 1m 9s	remaining: 51.7s
574:	learn: 1.3050177	total: 1m 9s	remaining: 51.6s
575:	learn: 1.3037514	total: 1m 9s	remaining: 51.4s
576:	learn: 1.3026627	total: 1m 9s	remaining: 51.3s
577:	learn: 1.3014802	total: 1m 10s	remaining: 51.2s
578:	learn: 1.2996921	total: 1m 10s	remaining: 51.1s
579:	learn: 1.2983376	total: 1m 10s	remaining: 51s
580:	learn: 1.2967458	total: 1m 10s	remaining: 50.8s
581:	learn: 1.2960582	total: 1m 10s	remaining: 50.7s
582:	learn:

719:	learn: 1.1337409	total: 1m 27s	remaining: 34s
720:	learn: 1.1328386	total: 1m 27s	remaining: 33.9s
721:	learn: 1.1319185	total: 1m 27s	remaining: 33.7s
722:	learn: 1.1307686	total: 1m 27s	remaining: 33.6s
723:	learn: 1.1295942	total: 1m 27s	remaining: 33.5s
724:	learn: 1.1281958	total: 1m 27s	remaining: 33.4s
725:	learn: 1.1270336	total: 1m 28s	remaining: 33.2s
726:	learn: 1.1267215	total: 1m 28s	remaining: 33.1s
727:	learn: 1.1259455	total: 1m 28s	remaining: 33s
728:	learn: 1.1239635	total: 1m 28s	remaining: 32.9s
729:	learn: 1.1235280	total: 1m 28s	remaining: 32.8s
730:	learn: 1.1220335	total: 1m 28s	remaining: 32.6s
731:	learn: 1.1217581	total: 1m 28s	remaining: 32.5s
732:	learn: 1.1203939	total: 1m 28s	remaining: 32.4s
733:	learn: 1.1202199	total: 1m 29s	remaining: 32.3s
734:	learn: 1.1192237	total: 1m 29s	remaining: 32.2s
735:	learn: 1.1182111	total: 1m 29s	remaining: 32s
736:	learn: 1.1158231	total: 1m 29s	remaining: 31.9s
737:	learn: 1.1151282	total: 1m 29s	remaining: 31.8s

875:	learn: 0.9997856	total: 1m 46s	remaining: 15s
876:	learn: 0.9986707	total: 1m 46s	remaining: 14.9s
877:	learn: 0.9980005	total: 1m 46s	remaining: 14.8s
878:	learn: 0.9969413	total: 1m 46s	remaining: 14.7s
879:	learn: 0.9959277	total: 1m 46s	remaining: 14.6s
880:	learn: 0.9948943	total: 1m 46s	remaining: 14.4s
881:	learn: 0.9934107	total: 1m 46s	remaining: 14.3s
882:	learn: 0.9929877	total: 1m 47s	remaining: 14.2s
883:	learn: 0.9919247	total: 1m 47s	remaining: 14.1s
884:	learn: 0.9914117	total: 1m 47s	remaining: 13.9s
885:	learn: 0.9906820	total: 1m 47s	remaining: 13.8s
886:	learn: 0.9897397	total: 1m 47s	remaining: 13.7s
887:	learn: 0.9883032	total: 1m 47s	remaining: 13.6s
888:	learn: 0.9880337	total: 1m 47s	remaining: 13.5s
889:	learn: 0.9866141	total: 1m 47s	remaining: 13.3s
890:	learn: 0.9852908	total: 1m 48s	remaining: 13.2s
891:	learn: 0.9848715	total: 1m 48s	remaining: 13.1s
892:	learn: 0.9846771	total: 1m 48s	remaining: 13s
893:	learn: 0.9837949	total: 1m 48s	remaining: 12.

In [17]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

1.282440626909335
1.2808983458860896
1.2908662982581671
1.2997237141231046
1.242504731644408
1.2982100446349043
0.7774063921508159
0.7743346692731723
0.7547951159353318
0.7554924343468408
0.8037302726795403
0.7986794685191938


In [18]:
train_size = 7000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)

Learning rate set to 0.055681
0:	learn: 5.0559179	total: 158ms	remaining: 2m 37s
1:	learn: 4.9338809	total: 287ms	remaining: 2m 23s
2:	learn: 4.8145470	total: 416ms	remaining: 2m 18s
3:	learn: 4.7022334	total: 545ms	remaining: 2m 15s
4:	learn: 4.5918923	total: 674ms	remaining: 2m 14s
5:	learn: 4.4929040	total: 803ms	remaining: 2m 12s
6:	learn: 4.4008001	total: 932ms	remaining: 2m 12s
7:	learn: 4.3198620	total: 1.06s	remaining: 2m 11s
8:	learn: 4.2379319	total: 1.19s	remaining: 2m 10s
9:	learn: 4.1560643	total: 1.32s	remaining: 2m 10s
10:	learn: 4.0784020	total: 1.45s	remaining: 2m 10s
11:	learn: 4.0082523	total: 1.58s	remaining: 2m 9s
12:	learn: 3.9439339	total: 1.71s	remaining: 2m 9s
13:	learn: 3.8780590	total: 1.83s	remaining: 2m 9s
14:	learn: 3.8180612	total: 1.96s	remaining: 2m 8s
15:	learn: 3.7672743	total: 2.09s	remaining: 2m 8s
16:	learn: 3.7132723	total: 2.22s	remaining: 2m 8s
17:	learn: 3.6639113	total: 2.35s	remaining: 2m 8s
18:	learn: 3.6133020	total: 2.48s	remaining: 2m 8s


159:	learn: 2.2313140	total: 20.5s	remaining: 1m 47s
160:	learn: 2.2300700	total: 20.6s	remaining: 1m 47s
161:	learn: 2.2262245	total: 20.7s	remaining: 1m 47s
162:	learn: 2.2192444	total: 20.8s	remaining: 1m 47s
163:	learn: 2.2163029	total: 21s	remaining: 1m 46s
164:	learn: 2.2113515	total: 21.1s	remaining: 1m 46s
165:	learn: 2.2103929	total: 21.2s	remaining: 1m 46s
166:	learn: 2.2046766	total: 21.3s	remaining: 1m 46s
167:	learn: 2.1972821	total: 21.5s	remaining: 1m 46s
168:	learn: 2.1938393	total: 21.6s	remaining: 1m 46s
169:	learn: 2.1892900	total: 21.7s	remaining: 1m 46s
170:	learn: 2.1849969	total: 21.9s	remaining: 1m 45s
171:	learn: 2.1819117	total: 22s	remaining: 1m 45s
172:	learn: 2.1786057	total: 22.1s	remaining: 1m 45s
173:	learn: 2.1761929	total: 22.2s	remaining: 1m 45s
174:	learn: 2.1753533	total: 22.4s	remaining: 1m 45s
175:	learn: 2.1702699	total: 22.5s	remaining: 1m 45s
176:	learn: 2.1694716	total: 22.6s	remaining: 1m 45s
177:	learn: 2.1657092	total: 22.7s	remaining: 1m 4

315:	learn: 1.7621531	total: 40.3s	remaining: 1m 27s
316:	learn: 1.7580237	total: 40.4s	remaining: 1m 27s
317:	learn: 1.7552448	total: 40.6s	remaining: 1m 27s
318:	learn: 1.7511759	total: 40.7s	remaining: 1m 26s
319:	learn: 1.7506213	total: 40.8s	remaining: 1m 26s
320:	learn: 1.7488368	total: 41s	remaining: 1m 26s
321:	learn: 1.7479502	total: 41.1s	remaining: 1m 26s
322:	learn: 1.7440262	total: 41.2s	remaining: 1m 26s
323:	learn: 1.7415918	total: 41.3s	remaining: 1m 26s
324:	learn: 1.7411647	total: 41.5s	remaining: 1m 26s
325:	learn: 1.7386891	total: 41.6s	remaining: 1m 25s
326:	learn: 1.7361998	total: 41.7s	remaining: 1m 25s
327:	learn: 1.7349151	total: 41.8s	remaining: 1m 25s
328:	learn: 1.7318788	total: 42s	remaining: 1m 25s
329:	learn: 1.7300374	total: 42.1s	remaining: 1m 25s
330:	learn: 1.7272426	total: 42.2s	remaining: 1m 25s
331:	learn: 1.7261855	total: 42.4s	remaining: 1m 25s
332:	learn: 1.7232994	total: 42.5s	remaining: 1m 25s
333:	learn: 1.7198390	total: 42.6s	remaining: 1m 2

471:	learn: 1.4677983	total: 1m	remaining: 1m 7s
472:	learn: 1.4675504	total: 1m	remaining: 1m 7s
473:	learn: 1.4659323	total: 1m	remaining: 1m 7s
474:	learn: 1.4637513	total: 1m	remaining: 1m 6s
475:	learn: 1.4635068	total: 1m	remaining: 1m 6s
476:	learn: 1.4616590	total: 1m	remaining: 1m 6s
477:	learn: 1.4597947	total: 1m	remaining: 1m 6s
478:	learn: 1.4595655	total: 1m 1s	remaining: 1m 6s
479:	learn: 1.4575989	total: 1m 1s	remaining: 1m 6s
480:	learn: 1.4555629	total: 1m 1s	remaining: 1m 6s
481:	learn: 1.4541510	total: 1m 1s	remaining: 1m 6s
482:	learn: 1.4524208	total: 1m 1s	remaining: 1m 5s
483:	learn: 1.4504819	total: 1m 1s	remaining: 1m 5s
484:	learn: 1.4502187	total: 1m 1s	remaining: 1m 5s
485:	learn: 1.4484286	total: 1m 1s	remaining: 1m 5s
486:	learn: 1.4469747	total: 1m 2s	remaining: 1m 5s
487:	learn: 1.4453574	total: 1m 2s	remaining: 1m 5s
488:	learn: 1.4444114	total: 1m 2s	remaining: 1m 5s
489:	learn: 1.4432858	total: 1m 2s	remaining: 1m 5s
490:	learn: 1.4416822	total: 1m 2

629:	learn: 1.2714351	total: 1m 20s	remaining: 47.2s
630:	learn: 1.2705559	total: 1m 20s	remaining: 47.1s
631:	learn: 1.2692387	total: 1m 20s	remaining: 46.9s
632:	learn: 1.2684885	total: 1m 20s	remaining: 46.8s
633:	learn: 1.2677583	total: 1m 20s	remaining: 46.7s
634:	learn: 1.2671355	total: 1m 20s	remaining: 46.6s
635:	learn: 1.2668422	total: 1m 21s	remaining: 46.4s
636:	learn: 1.2657882	total: 1m 21s	remaining: 46.3s
637:	learn: 1.2643372	total: 1m 21s	remaining: 46.2s
638:	learn: 1.2634271	total: 1m 21s	remaining: 46s
639:	learn: 1.2628972	total: 1m 21s	remaining: 45.9s
640:	learn: 1.2611363	total: 1m 21s	remaining: 45.8s
641:	learn: 1.2597442	total: 1m 21s	remaining: 45.7s
642:	learn: 1.2583280	total: 1m 22s	remaining: 45.5s
643:	learn: 1.2564015	total: 1m 22s	remaining: 45.4s
644:	learn: 1.2554834	total: 1m 22s	remaining: 45.3s
645:	learn: 1.2541620	total: 1m 22s	remaining: 45.1s
646:	learn: 1.2531830	total: 1m 22s	remaining: 45s
647:	learn: 1.2520407	total: 1m 22s	remaining: 44.

...

243:	learn: 1.9260603	total: 31.2s	remaining: 1m 36s
244:	learn: 1.9223765	total: 31.3s	remaining: 1m 36s
245:	learn: 1.9184984	total: 31.5s	remaining: 1m 36s
246:	learn: 1.9151815	total: 31.6s	remaining: 1m 36s
247:	learn: 1.9112859	total: 31.7s	remaining: 1m 36s
248:	learn: 1.9083881	total: 31.8s	remaining: 1m 36s
249:	learn: 1.9058676	total: 32s	remaining: 1m 35s
250:	learn: 1.9017354	total: 32.1s	remaining: 1m 35s
251:	learn: 1.9010334	total: 32.2s	remaining: 1m 35s
252:	learn: 1.8978840	total: 32.3s	remaining: 1m 35s
253:	learn: 1.8943847	total: 32.5s	remaining: 1m 35s
254:	learn: 1.8907744	total: 32.6s	remaining: 1m 35s
255:	learn: 1.8885263	total: 32.7s	remaining: 1m 35s
256:	learn: 1.8847276	total: 32.9s	remaining: 1m 34s
257:	learn: 1.8840412	total: 33s	remaining: 1m 34s
258:	learn: 1.8790902	total: 33.1s	remaining: 1m 34s
259:	learn: 1.8761361	total: 33.2s	remaining: 1m 34s
260:	learn: 1.8726401	total: 33.4s	remaining: 1m 34s
261:	learn: 1.8719540	total: 33.5s	remaining: 1m 3

399:	learn: 1.5686472	total: 51.1s	remaining: 1m 16s
400:	learn: 1.5661153	total: 51.2s	remaining: 1m 16s
401:	learn: 1.5651980	total: 51.4s	remaining: 1m 16s
402:	learn: 1.5632605	total: 51.5s	remaining: 1m 16s
403:	learn: 1.5627797	total: 51.6s	remaining: 1m 16s
404:	learn: 1.5608049	total: 51.7s	remaining: 1m 16s
405:	learn: 1.5583514	total: 51.9s	remaining: 1m 15s
406:	learn: 1.5568079	total: 52s	remaining: 1m 15s
407:	learn: 1.5553133	total: 52.1s	remaining: 1m 15s
408:	learn: 1.5522623	total: 52.3s	remaining: 1m 15s
409:	learn: 1.5506636	total: 52.4s	remaining: 1m 15s
410:	learn: 1.5471738	total: 52.5s	remaining: 1m 15s
411:	learn: 1.5455935	total: 52.6s	remaining: 1m 15s
412:	learn: 1.5433282	total: 52.8s	remaining: 1m 14s
413:	learn: 1.5419764	total: 52.9s	remaining: 1m 14s
414:	learn: 1.5396142	total: 53s	remaining: 1m 14s
415:	learn: 1.5355795	total: 53.1s	remaining: 1m 14s
416:	learn: 1.5349639	total: 53.3s	remaining: 1m 14s
417:	learn: 1.5335724	total: 53.4s	remaining: 1m 1

557:	learn: 1.3314830	total: 1m 11s	remaining: 56.4s
558:	learn: 1.3306680	total: 1m 11s	remaining: 56.3s
559:	learn: 1.3293283	total: 1m 11s	remaining: 56.2s
560:	learn: 1.3275135	total: 1m 11s	remaining: 56s
561:	learn: 1.3263381	total: 1m 11s	remaining: 55.9s
562:	learn: 1.3259906	total: 1m 11s	remaining: 55.8s
563:	learn: 1.3255670	total: 1m 11s	remaining: 55.7s
564:	learn: 1.3253437	total: 1m 12s	remaining: 55.5s
565:	learn: 1.3250109	total: 1m 12s	remaining: 55.4s
566:	learn: 1.3247700	total: 1m 12s	remaining: 55.3s
567:	learn: 1.3233799	total: 1m 12s	remaining: 55.1s
568:	learn: 1.3221396	total: 1m 12s	remaining: 55s
569:	learn: 1.3219102	total: 1m 12s	remaining: 54.9s
570:	learn: 1.3204015	total: 1m 12s	remaining: 54.8s
571:	learn: 1.3188326	total: 1m 13s	remaining: 54.6s
572:	learn: 1.3171074	total: 1m 13s	remaining: 54.5s
573:	learn: 1.3156911	total: 1m 13s	remaining: 54.4s
574:	learn: 1.3154719	total: 1m 13s	remaining: 54.2s
575:	learn: 1.3152644	total: 1m 13s	remaining: 54.

713:	learn: 1.1610684	total: 1m 31s	remaining: 36.5s
714:	learn: 1.1604539	total: 1m 31s	remaining: 36.4s
715:	learn: 1.1594677	total: 1m 31s	remaining: 36.2s
716:	learn: 1.1582345	total: 1m 31s	remaining: 36.1s
717:	learn: 1.1569961	total: 1m 31s	remaining: 36s
718:	learn: 1.1560687	total: 1m 31s	remaining: 35.9s
719:	learn: 1.1546170	total: 1m 31s	remaining: 35.7s
720:	learn: 1.1533606	total: 1m 32s	remaining: 35.6s
721:	learn: 1.1524368	total: 1m 32s	remaining: 35.5s
722:	learn: 1.1515513	total: 1m 32s	remaining: 35.3s
723:	learn: 1.1506337	total: 1m 32s	remaining: 35.2s
724:	learn: 1.1489306	total: 1m 32s	remaining: 35.1s
725:	learn: 1.1475789	total: 1m 32s	remaining: 35s
726:	learn: 1.1463130	total: 1m 32s	remaining: 34.8s
727:	learn: 1.1456285	total: 1m 32s	remaining: 34.7s
728:	learn: 1.1448070	total: 1m 33s	remaining: 34.6s
729:	learn: 1.1439682	total: 1m 33s	remaining: 34.5s
730:	learn: 1.1425695	total: 1m 33s	remaining: 34.3s
731:	learn: 1.1419133	total: 1m 33s	remaining: 34.

869:	learn: 1.0236129	total: 1m 51s	remaining: 16.6s
870:	learn: 1.0227231	total: 1m 51s	remaining: 16.5s
871:	learn: 1.0219860	total: 1m 51s	remaining: 16.3s
872:	learn: 1.0218044	total: 1m 51s	remaining: 16.2s
873:	learn: 1.0207177	total: 1m 51s	remaining: 16.1s
874:	learn: 1.0198179	total: 1m 51s	remaining: 16s
875:	learn: 1.0193141	total: 1m 51s	remaining: 15.8s
876:	learn: 1.0185022	total: 1m 51s	remaining: 15.7s
877:	learn: 1.0176322	total: 1m 52s	remaining: 15.6s
878:	learn: 1.0164572	total: 1m 52s	remaining: 15.4s
879:	learn: 1.0163471	total: 1m 52s	remaining: 15.3s
880:	learn: 1.0149112	total: 1m 52s	remaining: 15.2s
881:	learn: 1.0137045	total: 1m 52s	remaining: 15.1s
882:	learn: 1.0131336	total: 1m 52s	remaining: 14.9s
883:	learn: 1.0123035	total: 1m 52s	remaining: 14.8s
884:	learn: 1.0109784	total: 1m 52s	remaining: 14.7s
885:	learn: 1.0099716	total: 1m 53s	remaining: 14.6s
886:	learn: 1.0089655	total: 1m 53s	remaining: 14.4s
887:	learn: 1.0081698	total: 1m 53s	remaining: 1

In [19]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

1.2781277172810017
1.2890398798226126
1.2802083514432796
1.306116827096705
1.242041053299998
1.2732324747424133
0.7860914118420839
0.7921565738171484
0.7823024812807484
0.7563764814770187
0.8006497460331965
0.798971776602307


In [20]:
train_size = 8000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)

Learning rate set to 0.056868
0:	learn: 5.0771260	total: 170ms	remaining: 2m 49s
1:	learn: 4.9555787	total: 304ms	remaining: 2m 31s
2:	learn: 4.8321831	total: 438ms	remaining: 2m 25s
3:	learn: 4.7119379	total: 571ms	remaining: 2m 22s
4:	learn: 4.5970477	total: 705ms	remaining: 2m 20s
5:	learn: 4.4970337	total: 838ms	remaining: 2m 18s
6:	learn: 4.4038431	total: 972ms	remaining: 2m 17s
7:	learn: 4.3098117	total: 1.1s	remaining: 2m 17s
8:	learn: 4.2219536	total: 1.24s	remaining: 2m 16s
9:	learn: 4.1390081	total: 1.37s	remaining: 2m 15s
10:	learn: 4.0585917	total: 1.51s	remaining: 2m 15s
11:	learn: 3.9817463	total: 1.64s	remaining: 2m 15s
12:	learn: 3.9104206	total: 1.77s	remaining: 2m 14s
13:	learn: 3.8445432	total: 1.91s	remaining: 2m 14s
14:	learn: 3.7852508	total: 2.04s	remaining: 2m 14s
15:	learn: 3.7288541	total: 2.18s	remaining: 2m 14s
16:	learn: 3.6724484	total: 2.31s	remaining: 2m 13s
17:	learn: 3.6169923	total: 2.44s	remaining: 2m 13s
18:	learn: 3.5696360	total: 2.58s	remaining: 

159:	learn: 2.1986963	total: 21.2s	remaining: 1m 51s
160:	learn: 2.1976650	total: 21.4s	remaining: 1m 51s
161:	learn: 2.1946707	total: 21.5s	remaining: 1m 51s
162:	learn: 2.1904318	total: 21.6s	remaining: 1m 51s
163:	learn: 2.1841788	total: 21.8s	remaining: 1m 50s
164:	learn: 2.1816553	total: 21.9s	remaining: 1m 50s
165:	learn: 2.1806273	total: 22s	remaining: 1m 50s
166:	learn: 2.1770602	total: 22.2s	remaining: 1m 50s
167:	learn: 2.1744282	total: 22.3s	remaining: 1m 50s
168:	learn: 2.1706383	total: 22.4s	remaining: 1m 50s
169:	learn: 2.1643476	total: 22.6s	remaining: 1m 50s
170:	learn: 2.1605621	total: 22.7s	remaining: 1m 49s
171:	learn: 2.1589070	total: 22.8s	remaining: 1m 49s
172:	learn: 2.1561430	total: 22.9s	remaining: 1m 49s
173:	learn: 2.1527070	total: 23.1s	remaining: 1m 49s
174:	learn: 2.1490598	total: 23.2s	remaining: 1m 49s
175:	learn: 2.1475983	total: 23.3s	remaining: 1m 49s
176:	learn: 2.1402690	total: 23.5s	remaining: 1m 49s
177:	learn: 2.1353979	total: 23.6s	remaining: 1m

315:	learn: 1.7587274	total: 41.8s	remaining: 1m 30s
316:	learn: 1.7567370	total: 41.9s	remaining: 1m 30s
317:	learn: 1.7537095	total: 42.1s	remaining: 1m 30s
318:	learn: 1.7512956	total: 42.2s	remaining: 1m 30s
319:	learn: 1.7493959	total: 42.3s	remaining: 1m 29s
320:	learn: 1.7463596	total: 42.5s	remaining: 1m 29s
321:	learn: 1.7441574	total: 42.6s	remaining: 1m 29s
322:	learn: 1.7415965	total: 42.7s	remaining: 1m 29s
323:	learn: 1.7409872	total: 42.9s	remaining: 1m 29s
324:	learn: 1.7368929	total: 43s	remaining: 1m 29s
325:	learn: 1.7351774	total: 43.1s	remaining: 1m 29s
326:	learn: 1.7316404	total: 43.3s	remaining: 1m 29s
327:	learn: 1.7289246	total: 43.4s	remaining: 1m 28s
328:	learn: 1.7285822	total: 43.5s	remaining: 1m 28s
329:	learn: 1.7264244	total: 43.7s	remaining: 1m 28s
330:	learn: 1.7229843	total: 43.8s	remaining: 1m 28s
331:	learn: 1.7205751	total: 43.9s	remaining: 1m 28s
332:	learn: 1.7180735	total: 44.1s	remaining: 1m 28s
333:	learn: 1.7173282	total: 44.2s	remaining: 1m

471:	learn: 1.4794158	total: 1m 2s	remaining: 1m 9s
472:	learn: 1.4779820	total: 1m 2s	remaining: 1m 9s
473:	learn: 1.4764269	total: 1m 2s	remaining: 1m 9s
474:	learn: 1.4753504	total: 1m 2s	remaining: 1m 9s
475:	learn: 1.4737198	total: 1m 2s	remaining: 1m 9s
476:	learn: 1.4723903	total: 1m 3s	remaining: 1m 9s
477:	learn: 1.4708689	total: 1m 3s	remaining: 1m 9s
478:	learn: 1.4687314	total: 1m 3s	remaining: 1m 8s
479:	learn: 1.4683539	total: 1m 3s	remaining: 1m 8s
480:	learn: 1.4666737	total: 1m 3s	remaining: 1m 8s
481:	learn: 1.4653634	total: 1m 3s	remaining: 1m 8s
482:	learn: 1.4642262	total: 1m 3s	remaining: 1m 8s
483:	learn: 1.4632483	total: 1m 4s	remaining: 1m 8s
484:	learn: 1.4617606	total: 1m 4s	remaining: 1m 8s
485:	learn: 1.4615257	total: 1m 4s	remaining: 1m 7s
486:	learn: 1.4601210	total: 1m 4s	remaining: 1m 7s
487:	learn: 1.4587699	total: 1m 4s	remaining: 1m 7s
488:	learn: 1.4571476	total: 1m 4s	remaining: 1m 7s
489:	learn: 1.4553937	total: 1m 4s	remaining: 1m 7s
490:	learn: 

629:	learn: 1.2736467	total: 1m 23s	remaining: 48.9s
630:	learn: 1.2720509	total: 1m 23s	remaining: 48.8s
631:	learn: 1.2709108	total: 1m 23s	remaining: 48.7s
632:	learn: 1.2705191	total: 1m 23s	remaining: 48.5s
633:	learn: 1.2693193	total: 1m 23s	remaining: 48.4s
634:	learn: 1.2677884	total: 1m 24s	remaining: 48.3s
635:	learn: 1.2663154	total: 1m 24s	remaining: 48.2s
636:	learn: 1.2649176	total: 1m 24s	remaining: 48s
637:	learn: 1.2642626	total: 1m 24s	remaining: 47.9s
638:	learn: 1.2620597	total: 1m 24s	remaining: 47.8s
639:	learn: 1.2601769	total: 1m 24s	remaining: 47.6s
640:	learn: 1.2589127	total: 1m 24s	remaining: 47.5s
641:	learn: 1.2578116	total: 1m 24s	remaining: 47.4s
642:	learn: 1.2566147	total: 1m 25s	remaining: 47.2s
643:	learn: 1.2556997	total: 1m 25s	remaining: 47.1s
644:	learn: 1.2538845	total: 1m 25s	remaining: 47s
645:	learn: 1.2526932	total: 1m 25s	remaining: 46.8s
646:	learn: 1.2506492	total: 1m 25s	remaining: 46.7s
647:	learn: 1.2499788	total: 1m 25s	remaining: 46.

...

245:	learn: 1.9603051	total: 32.6s	remaining: 1m 40s
246:	learn: 1.9595021	total: 32.8s	remaining: 1m 39s
247:	learn: 1.9555625	total: 32.9s	remaining: 1m 39s
248:	learn: 1.9528846	total: 33s	remaining: 1m 39s
249:	learn: 1.9520517	total: 33.2s	remaining: 1m 39s
250:	learn: 1.9501449	total: 33.3s	remaining: 1m 39s
251:	learn: 1.9472756	total: 33.4s	remaining: 1m 39s
252:	learn: 1.9434683	total: 33.6s	remaining: 1m 39s
253:	learn: 1.9404715	total: 33.7s	remaining: 1m 38s
254:	learn: 1.9360679	total: 33.8s	remaining: 1m 38s
255:	learn: 1.9324073	total: 34s	remaining: 1m 38s
256:	learn: 1.9301445	total: 34.1s	remaining: 1m 38s
257:	learn: 1.9275872	total: 34.2s	remaining: 1m 38s
258:	learn: 1.9243793	total: 34.4s	remaining: 1m 38s
259:	learn: 1.9236077	total: 34.5s	remaining: 1m 38s
260:	learn: 1.9215287	total: 34.6s	remaining: 1m 38s
261:	learn: 1.9190397	total: 34.8s	remaining: 1m 37s
262:	learn: 1.9165069	total: 34.9s	remaining: 1m 37s
263:	learn: 1.9145426	total: 35s	remaining: 1m 37s

401:	learn: 1.6242078	total: 53.3s	remaining: 1m 19s
402:	learn: 1.6232922	total: 53.4s	remaining: 1m 19s
403:	learn: 1.6217928	total: 53.6s	remaining: 1m 19s
404:	learn: 1.6193237	total: 53.7s	remaining: 1m 18s
405:	learn: 1.6175055	total: 53.8s	remaining: 1m 18s
406:	learn: 1.6147470	total: 54s	remaining: 1m 18s
407:	learn: 1.6142775	total: 54.1s	remaining: 1m 18s
408:	learn: 1.6118407	total: 54.2s	remaining: 1m 18s
409:	learn: 1.6106023	total: 54.4s	remaining: 1m 18s
410:	learn: 1.6087267	total: 54.5s	remaining: 1m 18s
411:	learn: 1.6084180	total: 54.6s	remaining: 1m 17s
412:	learn: 1.6068055	total: 54.8s	remaining: 1m 17s
413:	learn: 1.6052872	total: 54.9s	remaining: 1m 17s
414:	learn: 1.6034142	total: 55s	remaining: 1m 17s
415:	learn: 1.6018618	total: 55.2s	remaining: 1m 17s
416:	learn: 1.6007621	total: 55.3s	remaining: 1m 17s
417:	learn: 1.5996859	total: 55.4s	remaining: 1m 17s
418:	learn: 1.5985165	total: 55.5s	remaining: 1m 17s
419:	learn: 1.5975014	total: 55.7s	remaining: 1m 1

559:	learn: 1.3873257	total: 1m 14s	remaining: 58.3s
560:	learn: 1.3858827	total: 1m 14s	remaining: 58.2s
561:	learn: 1.3847713	total: 1m 14s	remaining: 58.1s
562:	learn: 1.3831142	total: 1m 14s	remaining: 57.9s
563:	learn: 1.3822382	total: 1m 14s	remaining: 57.8s
564:	learn: 1.3805958	total: 1m 14s	remaining: 57.7s
565:	learn: 1.3781446	total: 1m 15s	remaining: 57.5s
566:	learn: 1.3768936	total: 1m 15s	remaining: 57.4s
567:	learn: 1.3753025	total: 1m 15s	remaining: 57.3s
568:	learn: 1.3741579	total: 1m 15s	remaining: 57.1s
569:	learn: 1.3726809	total: 1m 15s	remaining: 57s
570:	learn: 1.3715761	total: 1m 15s	remaining: 56.9s
571:	learn: 1.3696435	total: 1m 15s	remaining: 56.8s
572:	learn: 1.3686559	total: 1m 15s	remaining: 56.6s
573:	learn: 1.3674788	total: 1m 16s	remaining: 56.5s
574:	learn: 1.3666553	total: 1m 16s	remaining: 56.4s
575:	learn: 1.3644112	total: 1m 16s	remaining: 56.2s
576:	learn: 1.3630398	total: 1m 16s	remaining: 56.1s
577:	learn: 1.3627916	total: 1m 16s	remaining: 5

715:	learn: 1.2114996	total: 1m 34s	remaining: 37.7s
716:	learn: 1.2110943	total: 1m 35s	remaining: 37.5s
717:	learn: 1.2095775	total: 1m 35s	remaining: 37.4s
718:	learn: 1.2082919	total: 1m 35s	remaining: 37.3s
719:	learn: 1.2074763	total: 1m 35s	remaining: 37.1s
720:	learn: 1.2072385	total: 1m 35s	remaining: 37s
721:	learn: 1.2064287	total: 1m 35s	remaining: 36.9s
722:	learn: 1.2053845	total: 1m 35s	remaining: 36.7s
723:	learn: 1.2048203	total: 1m 36s	remaining: 36.6s
724:	learn: 1.2035397	total: 1m 36s	remaining: 36.5s
725:	learn: 1.2023625	total: 1m 36s	remaining: 36.3s
726:	learn: 1.2013145	total: 1m 36s	remaining: 36.2s
727:	learn: 1.1994093	total: 1m 36s	remaining: 36.1s
728:	learn: 1.1980448	total: 1m 36s	remaining: 35.9s
729:	learn: 1.1966473	total: 1m 36s	remaining: 35.8s
730:	learn: 1.1962694	total: 1m 36s	remaining: 35.7s
731:	learn: 1.1948456	total: 1m 37s	remaining: 35.5s
732:	learn: 1.1946047	total: 1m 37s	remaining: 35.4s
733:	learn: 1.1938201	total: 1m 37s	remaining: 3

871:	learn: 1.0740632	total: 1m 55s	remaining: 17s
872:	learn: 1.0733296	total: 1m 55s	remaining: 16.8s
873:	learn: 1.0730058	total: 1m 55s	remaining: 16.7s
874:	learn: 1.0722505	total: 1m 56s	remaining: 16.6s
875:	learn: 1.0706600	total: 1m 56s	remaining: 16.4s
876:	learn: 1.0705462	total: 1m 56s	remaining: 16.3s
877:	learn: 1.0695404	total: 1m 56s	remaining: 16.2s
878:	learn: 1.0685841	total: 1m 56s	remaining: 16s
879:	learn: 1.0675613	total: 1m 56s	remaining: 15.9s
880:	learn: 1.0661275	total: 1m 56s	remaining: 15.8s
881:	learn: 1.0659456	total: 1m 56s	remaining: 15.6s
882:	learn: 1.0649589	total: 1m 57s	remaining: 15.5s
883:	learn: 1.0641540	total: 1m 57s	remaining: 15.4s
884:	learn: 1.0632551	total: 1m 57s	remaining: 15.2s
885:	learn: 1.0623701	total: 1m 57s	remaining: 15.1s
886:	learn: 1.0615696	total: 1m 57s	remaining: 15s
887:	learn: 1.0606686	total: 1m 57s	remaining: 14.9s
888:	learn: 1.0599202	total: 1m 57s	remaining: 14.7s
889:	learn: 1.0593294	total: 1m 58s	remaining: 14.6s

In [21]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

1.2630122654902323
1.3081443447206573
1.2378798132037392
1.2838182080605511
1.2265440243195647
1.2586749371466497
0.7873694104008004
0.7893204239704001
0.790608446059444
0.7559119413082537
0.8109004720102365
0.7901057686556681


In [22]:
train_size = 9000

X_train_0, X_test_0, y_train_0, y_test_0 = train_test_split(X, y, train_size = train_size, random_state=0)

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y, train_size = train_size, random_state=1)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y, train_size = train_size, random_state=2)

X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, train_size = train_size, random_state=3)

X_train_4, X_test_4, y_train_4, y_test_4 = train_test_split(X, y, train_size = train_size, random_state=4)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_0,y_train_0)
y_pred_0 = model.predict(X_test_0)
loss_0 = mean_absolute_error(y_test_0,y_pred_0)
accuracy_0 = r2_score(y_test_0,y_pred_0)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_1,y_train_1)
y_pred_1 = model.predict(X_test_1)
loss_1 = mean_absolute_error(y_test_1,y_pred_1)
accuracy_1 = r2_score(y_test_1,y_pred_1)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_2,y_train_2)
y_pred_2 = model.predict(X_test_2)
loss_2 = mean_absolute_error(y_test_2,y_pred_2)
accuracy_2 = r2_score(y_test_2,y_pred_2)


model = CatBoostRegressor(random_state=42)
model.fit(X_train_3,y_train_3)
y_pred_3 = model.predict(X_test_3)
loss_3 = mean_absolute_error(y_test_3,y_pred_3)
accuracy_3 = r2_score(y_test_3,y_pred_3)

model = CatBoostRegressor(random_state=42)
model.fit(X_train_4,y_train_4)
y_pred_4 = model.predict(X_test_4)
loss_4 = mean_absolute_error(y_test_4,y_pred_4)
accuracy_4 = r2_score(y_test_4,y_pred_4)

Learning rate set to 0.057936
0:	learn: 5.1871913	total: 174ms	remaining: 2m 54s
1:	learn: 5.0549699	total: 312ms	remaining: 2m 35s
2:	learn: 4.9236957	total: 452ms	remaining: 2m 30s
3:	learn: 4.8009373	total: 590ms	remaining: 2m 26s
4:	learn: 4.6819954	total: 728ms	remaining: 2m 24s
5:	learn: 4.5673549	total: 865ms	remaining: 2m 23s
6:	learn: 4.4625360	total: 1s	remaining: 2m 22s
7:	learn: 4.3599018	total: 1.14s	remaining: 2m 21s
8:	learn: 4.2668721	total: 1.28s	remaining: 2m 20s
9:	learn: 4.1785354	total: 1.42s	remaining: 2m 20s
10:	learn: 4.0975423	total: 1.56s	remaining: 2m 20s
11:	learn: 4.0242357	total: 1.7s	remaining: 2m 19s
12:	learn: 3.9575521	total: 1.84s	remaining: 2m 19s
13:	learn: 3.8904234	total: 1.98s	remaining: 2m 19s
14:	learn: 3.8260788	total: 2.12s	remaining: 2m 18s
15:	learn: 3.7657959	total: 2.25s	remaining: 2m 18s
16:	learn: 3.7112380	total: 2.39s	remaining: 2m 18s
17:	learn: 3.6543159	total: 2.53s	remaining: 2m 18s
18:	learn: 3.5998621	total: 2.67s	remaining: 2m 

159:	learn: 2.2167036	total: 22s	remaining: 1m 55s
160:	learn: 2.2148730	total: 22.1s	remaining: 1m 55s
161:	learn: 2.2108494	total: 22.3s	remaining: 1m 55s
162:	learn: 2.2099297	total: 22.4s	remaining: 1m 55s
163:	learn: 2.2049593	total: 22.6s	remaining: 1m 54s
164:	learn: 2.2007769	total: 22.7s	remaining: 1m 54s
165:	learn: 2.1930960	total: 22.8s	remaining: 1m 54s
166:	learn: 2.1892476	total: 23s	remaining: 1m 54s
167:	learn: 2.1878143	total: 23.1s	remaining: 1m 54s
168:	learn: 2.1826786	total: 23.2s	remaining: 1m 54s
169:	learn: 2.1798789	total: 23.4s	remaining: 1m 54s
170:	learn: 2.1768780	total: 23.5s	remaining: 1m 53s
171:	learn: 2.1756146	total: 23.6s	remaining: 1m 53s
172:	learn: 2.1709759	total: 23.8s	remaining: 1m 53s
173:	learn: 2.1669909	total: 23.9s	remaining: 1m 53s
174:	learn: 2.1648917	total: 24.1s	remaining: 1m 53s
175:	learn: 2.1614615	total: 24.2s	remaining: 1m 53s
176:	learn: 2.1564260	total: 24.3s	remaining: 1m 53s
177:	learn: 2.1528404	total: 24.5s	remaining: 1m 5

315:	learn: 1.7790540	total: 43.4s	remaining: 1m 33s
316:	learn: 1.7767359	total: 43.5s	remaining: 1m 33s
317:	learn: 1.7763643	total: 43.6s	remaining: 1m 33s
318:	learn: 1.7735681	total: 43.8s	remaining: 1m 33s
319:	learn: 1.7706575	total: 43.9s	remaining: 1m 33s
320:	learn: 1.7700121	total: 44s	remaining: 1m 33s
321:	learn: 1.7672194	total: 44.2s	remaining: 1m 33s
322:	learn: 1.7648038	total: 44.3s	remaining: 1m 32s
323:	learn: 1.7643422	total: 44.4s	remaining: 1m 32s
324:	learn: 1.7620432	total: 44.6s	remaining: 1m 32s
325:	learn: 1.7596756	total: 44.7s	remaining: 1m 32s
326:	learn: 1.7569898	total: 44.9s	remaining: 1m 32s
327:	learn: 1.7540860	total: 45s	remaining: 1m 32s
328:	learn: 1.7517600	total: 45.1s	remaining: 1m 32s
329:	learn: 1.7494584	total: 45.3s	remaining: 1m 31s
330:	learn: 1.7491114	total: 45.4s	remaining: 1m 31s
331:	learn: 1.7465686	total: 45.5s	remaining: 1m 31s
332:	learn: 1.7439311	total: 45.7s	remaining: 1m 31s
333:	learn: 1.7419981	total: 45.8s	remaining: 1m 3

471:	learn: 1.5039871	total: 1m 4s	remaining: 1m 12s
472:	learn: 1.5012045	total: 1m 4s	remaining: 1m 12s
473:	learn: 1.4994736	total: 1m 5s	remaining: 1m 12s
474:	learn: 1.4992406	total: 1m 5s	remaining: 1m 12s
475:	learn: 1.4969468	total: 1m 5s	remaining: 1m 11s
476:	learn: 1.4964089	total: 1m 5s	remaining: 1m 11s
477:	learn: 1.4948553	total: 1m 5s	remaining: 1m 11s
478:	learn: 1.4929651	total: 1m 5s	remaining: 1m 11s
479:	learn: 1.4919062	total: 1m 5s	remaining: 1m 11s
480:	learn: 1.4901168	total: 1m 5s	remaining: 1m 11s
481:	learn: 1.4897978	total: 1m 6s	remaining: 1m 11s
482:	learn: 1.4885024	total: 1m 6s	remaining: 1m 10s
483:	learn: 1.4870579	total: 1m 6s	remaining: 1m 10s
484:	learn: 1.4852049	total: 1m 6s	remaining: 1m 10s
485:	learn: 1.4833703	total: 1m 6s	remaining: 1m 10s
486:	learn: 1.4815877	total: 1m 6s	remaining: 1m 10s
487:	learn: 1.4796368	total: 1m 6s	remaining: 1m 10s
488:	learn: 1.4780646	total: 1m 7s	remaining: 1m 10s
489:	learn: 1.4775761	total: 1m 7s	remaining: 

627:	learn: 1.3047681	total: 1m 26s	remaining: 51s
628:	learn: 1.3041366	total: 1m 26s	remaining: 50.9s
629:	learn: 1.3038277	total: 1m 26s	remaining: 50.8s
630:	learn: 1.3022561	total: 1m 26s	remaining: 50.6s
631:	learn: 1.3008664	total: 1m 26s	remaining: 50.5s
632:	learn: 1.3001177	total: 1m 26s	remaining: 50.3s
633:	learn: 1.2991343	total: 1m 26s	remaining: 50.2s
634:	learn: 1.2988428	total: 1m 27s	remaining: 50.1s
635:	learn: 1.2975838	total: 1m 27s	remaining: 49.9s
636:	learn: 1.2967588	total: 1m 27s	remaining: 49.8s
637:	learn: 1.2958323	total: 1m 27s	remaining: 49.7s
638:	learn: 1.2951333	total: 1m 27s	remaining: 49.5s
639:	learn: 1.2938554	total: 1m 27s	remaining: 49.4s
640:	learn: 1.2931929	total: 1m 27s	remaining: 49.2s
641:	learn: 1.2917771	total: 1m 28s	remaining: 49.1s
642:	learn: 1.2908096	total: 1m 28s	remaining: 49s
643:	learn: 1.2894743	total: 1m 28s	remaining: 48.8s
644:	learn: 1.2885465	total: 1m 28s	remaining: 48.7s
645:	learn: 1.2878007	total: 1m 28s	remaining: 48.

...

237:	learn: 1.9486399	total: 32.7s	remaining: 1m 44s
238:	learn: 1.9480639	total: 32.9s	remaining: 1m 44s
239:	learn: 1.9444675	total: 33s	remaining: 1m 44s
240:	learn: 1.9416011	total: 33.1s	remaining: 1m 44s
241:	learn: 1.9378477	total: 33.3s	remaining: 1m 44s
242:	learn: 1.9373383	total: 33.4s	remaining: 1m 44s
243:	learn: 1.9348724	total: 33.5s	remaining: 1m 43s
244:	learn: 1.9331484	total: 33.7s	remaining: 1m 43s
245:	learn: 1.9296428	total: 33.8s	remaining: 1m 43s
246:	learn: 1.9264377	total: 33.9s	remaining: 1m 43s
247:	learn: 1.9248191	total: 34.1s	remaining: 1m 43s
248:	learn: 1.9220741	total: 34.2s	remaining: 1m 43s
249:	learn: 1.9215925	total: 34.4s	remaining: 1m 43s
250:	learn: 1.9189559	total: 34.5s	remaining: 1m 42s
251:	learn: 1.9172294	total: 34.6s	remaining: 1m 42s
252:	learn: 1.9140975	total: 34.8s	remaining: 1m 42s
253:	learn: 1.9114291	total: 34.9s	remaining: 1m 42s
254:	learn: 1.9108149	total: 35s	remaining: 1m 42s
255:	learn: 1.9080865	total: 35.2s	remaining: 1m 4

393:	learn: 1.6276488	total: 54.1s	remaining: 1m 23s
394:	learn: 1.6249721	total: 54.2s	remaining: 1m 23s
395:	learn: 1.6228964	total: 54.4s	remaining: 1m 22s
396:	learn: 1.6206663	total: 54.5s	remaining: 1m 22s
397:	learn: 1.6179857	total: 54.6s	remaining: 1m 22s
398:	learn: 1.6162846	total: 54.8s	remaining: 1m 22s
399:	learn: 1.6140133	total: 54.9s	remaining: 1m 22s
400:	learn: 1.6112264	total: 55.1s	remaining: 1m 22s
401:	learn: 1.6109380	total: 55.2s	remaining: 1m 22s
402:	learn: 1.6092664	total: 55.3s	remaining: 1m 21s
403:	learn: 1.6070114	total: 55.5s	remaining: 1m 21s
404:	learn: 1.6057590	total: 55.6s	remaining: 1m 21s
405:	learn: 1.6040171	total: 55.7s	remaining: 1m 21s
406:	learn: 1.6026612	total: 55.9s	remaining: 1m 21s
407:	learn: 1.6007967	total: 56s	remaining: 1m 21s
408:	learn: 1.6003088	total: 56.1s	remaining: 1m 21s
409:	learn: 1.5991036	total: 56.3s	remaining: 1m 20s
410:	learn: 1.5963094	total: 56.4s	remaining: 1m 20s
411:	learn: 1.5954822	total: 56.6s	remaining: 1m

549:	learn: 1.4026310	total: 1m 15s	remaining: 1m 1s
550:	learn: 1.4010015	total: 1m 15s	remaining: 1m 1s
551:	learn: 1.4001174	total: 1m 15s	remaining: 1m 1s
552:	learn: 1.3990089	total: 1m 15s	remaining: 1m 1s
553:	learn: 1.3985747	total: 1m 16s	remaining: 1m 1s
554:	learn: 1.3981757	total: 1m 16s	remaining: 1m 1s
555:	learn: 1.3977555	total: 1m 16s	remaining: 1m
556:	learn: 1.3966615	total: 1m 16s	remaining: 1m
557:	learn: 1.3961833	total: 1m 16s	remaining: 1m
558:	learn: 1.3947975	total: 1m 16s	remaining: 1m
559:	learn: 1.3935419	total: 1m 16s	remaining: 1m
560:	learn: 1.3932024	total: 1m 16s	remaining: 1m
561:	learn: 1.3922104	total: 1m 17s	remaining: 1m
562:	learn: 1.3907664	total: 1m 17s	remaining: 60s
563:	learn: 1.3899170	total: 1m 17s	remaining: 59.8s
564:	learn: 1.3887064	total: 1m 17s	remaining: 59.7s
565:	learn: 1.3876539	total: 1m 17s	remaining: 59.5s
566:	learn: 1.3866728	total: 1m 17s	remaining: 59.4s
567:	learn: 1.3856144	total: 1m 17s	remaining: 59.3s
568:	learn: 1.38

705:	learn: 1.2341598	total: 1m 36s	remaining: 40.3s
706:	learn: 1.2332141	total: 1m 37s	remaining: 40.2s
707:	learn: 1.2316187	total: 1m 37s	remaining: 40.1s
708:	learn: 1.2296904	total: 1m 37s	remaining: 39.9s
709:	learn: 1.2281590	total: 1m 37s	remaining: 39.8s
710:	learn: 1.2271196	total: 1m 37s	remaining: 39.7s
711:	learn: 1.2262659	total: 1m 37s	remaining: 39.5s
712:	learn: 1.2254449	total: 1m 37s	remaining: 39.4s
713:	learn: 1.2246738	total: 1m 37s	remaining: 39.2s
714:	learn: 1.2245192	total: 1m 38s	remaining: 39.1s
715:	learn: 1.2236284	total: 1m 38s	remaining: 39s
716:	learn: 1.2227541	total: 1m 38s	remaining: 38.8s
717:	learn: 1.2213850	total: 1m 38s	remaining: 38.7s
718:	learn: 1.2201566	total: 1m 38s	remaining: 38.6s
719:	learn: 1.2191367	total: 1m 38s	remaining: 38.4s
720:	learn: 1.2172875	total: 1m 38s	remaining: 38.3s
721:	learn: 1.2161053	total: 1m 39s	remaining: 38.1s
722:	learn: 1.2151357	total: 1m 39s	remaining: 38s
723:	learn: 1.2142173	total: 1m 39s	remaining: 37.

861:	learn: 1.0977057	total: 1m 58s	remaining: 18.9s
862:	learn: 1.0972012	total: 1m 58s	remaining: 18.8s
863:	learn: 1.0964420	total: 1m 58s	remaining: 18.7s
864:	learn: 1.0959335	total: 1m 58s	remaining: 18.5s
865:	learn: 1.0954021	total: 1m 58s	remaining: 18.4s
866:	learn: 1.0948491	total: 1m 58s	remaining: 18.3s
867:	learn: 1.0946288	total: 1m 59s	remaining: 18.1s
868:	learn: 1.0937306	total: 1m 59s	remaining: 18s
869:	learn: 1.0931873	total: 1m 59s	remaining: 17.8s
870:	learn: 1.0927075	total: 1m 59s	remaining: 17.7s
871:	learn: 1.0917911	total: 1m 59s	remaining: 17.6s
872:	learn: 1.0907328	total: 1m 59s	remaining: 17.4s
873:	learn: 1.0905406	total: 1m 59s	remaining: 17.3s
874:	learn: 1.0898070	total: 2m	remaining: 17.2s
875:	learn: 1.0888878	total: 2m	remaining: 17s
876:	learn: 1.0880809	total: 2m	remaining: 16.9s
877:	learn: 1.0877450	total: 2m	remaining: 16.7s
878:	learn: 1.0865709	total: 2m	remaining: 16.6s
879:	learn: 1.0858890	total: 2m	remaining: 16.5s
880:	learn: 1.0853311

In [23]:
print((loss_0 + loss_1 + loss_2 + loss_3 + loss_4)/5)
print(loss_0)
print(loss_1)
print(loss_2)
print(loss_3)
print(loss_4)

print((accuracy_0 + accuracy_1 + accuracy_2 + accuracy_3 + accuracy_4)/5)
print(accuracy_0)
print(accuracy_1)
print(accuracy_2)
print(accuracy_3)
print(accuracy_4)

1.2350264379607814
1.277654630657797
1.1854384285510449
1.2043060012708746
1.2082916031356656
1.299441526188525
0.8190591067795336
0.8177657407988687
0.8123908572339856
0.8260611385815974
0.8331151869150857
0.8059626103681315
